# Running datasets through models. Inputs and output file names need changed in 2,3 & 7

In [ ]:
import pandas as pd
from fractions import Fraction

df = pd.read_csv('DS7_Calc_Tox_clean.csv')

def frac_str_to_float(frac_str):
    try:
        # Handle if already float or int
        if pd.isna(frac_str):
            return frac_str  # keep NaN as is
        if isinstance(frac_str, (float, int)):
            return frac_str
        return float(Fraction(frac_str))
    except Exception as e:
        print(f"Conversion error for value '{frac_str}': {e}")
        return None  # or np.nan

# Convert the fraction columns
df['Toxicophore_Presence'] = df['Toxicophore_Presence'].apply(frac_str_to_float)
df['Alerting_Structural_Motifs'] = df['Alerting_Structural_Motifs'].apply(frac_str_to_float)

# Check if any NaNs remain after conversion
print(df[['Toxicophore_Presence', 'Alerting_Structural_Motifs']].isna().sum())

# Save the fixed DataFrame back to the same CSV (overwrite)
df.to_csv('DS7_Calc_Tox_clean.csv', index=False)

print("✅ Conversion complete and saved to original CSV.")

import pandas as pd

# Load your original CSV
df = pd.read_csv('DS7_Calc_Tox_clean.csv')

# Filter out rows where Toxicity is 'Inconclusive' (keep only 'Active' or 'Inactive')
df_cleaned = df[df['Toxicity'].isin(['Active', 'Inactive'])].copy()

# Save the cleaned DataFrame back to CSV (overwrite or specify a new file)
df_cleaned.to_csv('DS7_Calc_Tox_clean.csv', index=False)

print(f"Rows with 'Inconclusive' Toxicity removed. Cleaned data saved.")

In [ ]:
#Updated file path
import numpy as np
import pandas as pd
file_path = 'DS7_Calc_Tox_clean.csv'
df1 = pd.read_csv(file_path)

# Prepare the features (E-state indices) and target (Activity_Outcomes)
X = df1[['Molecular_Weight', 'LogP', 'H_Acceptors', 'H_Donors', 'TPSA', 'Rotable_Bonds', 'Rings', 'Aromatic_Rings', 'SP3_Carbon', 'Toxicophore_Presence', 'Alerting_Structural_Motifs', 'QED', 'Lipinskis_Rule', 'Molar_Refractivity', 'Atoms_With_High_Ionization_Potential']]  # Features 

# Map 'Active' to 1 and 'Inactive' to 0
y = df1['Toxicity'].map({'Active': 1, 'Inactive': 0})


In [ ]:
print(df1.isna().sum())

In [ ]:
from sklearn.model_selection import train_test_split

# Split the data into training (80%) and testing (20%) sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Print the shapes of the resulting datasets
print(f"Training set size: {X_train.shape[0]} samples")
print(f"Testing set size: {X_test.shape[0]} samples")

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE

# Step 1: Split data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Step 2: Standard scale features (fit on training only)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Step 3: Apply SMOTE to only training data
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_scaled, y_train)

# Optional: Print shapes
print(f"Original X_train shape: {X_train.shape}")
print(f"Resampled X_train shape: {X_train_resampled.shape}")
print(f"X_test shape (unchanged): {X_test.shape}")


In [ ]:
# Update paths
# Paths
output_csv_path = r'C:\PhD\Prompt_Engineering\Outputs2\Precision_DS7_calc.csv'
feature_importance_csv = r'C:\PhD\Prompt_Engineering\Outputs2\Feature_Importance_DS7_calc.csv'

# Your specific feature list
feature_names = [
    'Molecular_Weight', 'LogP', 'H_Acceptors', 'H_Donors', 'TPSA',
    'Rotable_Bonds', 'Rings', 'Aromatic_Rings', 'SP3_Carbon',
    'Toxicophore_Presence', 'Alerting_Structural_Motifs', 'QED',
    'Lipinskis_Rule', 'Molar_Refractivity', 'Atoms_With_High_Ionization_Potential'
]

y_test_input = y_test


In [ ]:
#standard SVM -

import csv
import os
import pandas as pd
from sklearn.metrics import (
    accuracy_score, recall_score, confusion_matrix,
    roc_auc_score, matthews_corrcoef
)
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.inspection import permutation_importance
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import f1_score



model_title = "SVM_original"  # Change this title for each run

# Step 1: Train the SVM
svm_model = SVC(kernel='rbf', random_state=42, class_weight='balanced', probability=True)
svm_model.fit(X_train, y_train)

# Step 2: Predict
y_pred = svm_model.predict(X_test)
y_proba = svm_model.predict_proba(X_test)[:, 1]

# Step 3: Metrics
accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred, pos_label=1)
mcc = matthews_corrcoef(y_test, y_pred)
auc = roc_auc_score(y_test, y_proba)
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
specificity = tn / (tn + fp)
f1 = f1_score(y_test_input, y_pred)


# Step 4: Cross-validated recall
cv_recall_scores = cross_val_score(
    svm_model, X_train, y_train, cv=5, scoring='recall'
)
cv_recall_mean = cv_recall_scores.mean()

# Step 5: Save results to CSV
header = [
    "Model_Title", "Accuracy", "Recall", "Specificity",
    "AUC", "MCC", "CV_Recall_Mean", "F1"
]

row = [
    model_title, accuracy, recall, specificity,
    auc, mcc, cv_recall_mean, f1
]

# Check if file exists to write header only once
file_exists = os.path.isfile(output_csv_path)

with open(output_csv_path, mode='a', newline='') as file:
    writer = csv.writer(file)
    if not file_exists:
        writer.writerow(header)
    writer.writerow(row)

print(f"✅ Results saved to {output_csv_path}")

# --- Step 5: Permutation feature importance
print(f"\n📊 Computing feature importances for: {model_title}")
result = permutation_importance(
    svm_model, X_test, y_test, n_repeats=10, random_state=42, scoring='recall'
)
importances = pd.Series(result.importances_mean, index=feature_names)

# 🔄 Transpose: model title as row, features as columns
importance_df = pd.DataFrame(
    importances.values.reshape(1, -1), columns=feature_names, index=[model_title]
)

if os.path.exists(feature_importance_csv):
    existing_df = pd.read_csv(feature_importance_csv, index_col=0)
    updated_df = pd.concat([existing_df, importance_df])
else:
    updated_df = importance_df

updated_df.to_csv(feature_importance_csv)

print(f"✅ Feature importances saved to {feature_importance_csv}")

In [ ]:
#scaled SVM -

import csv
import os
import pandas as pd
from sklearn.metrics import (
    accuracy_score, recall_score, confusion_matrix,
    roc_auc_score, matthews_corrcoef
)
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.inspection import permutation_importance
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC


model_title = "SVM_scaled"  # Change this title for each run

# Step 1: Train the SVM
svm_model = SVC(kernel='rbf', random_state=42, class_weight='balanced', probability=True)
svm_model.fit(pd.DataFrame(X_train_scaled, columns=feature_names), y_train)


# Step 2: Predict
X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=feature_names)
y_pred = svm_model.predict(X_test_scaled_df)
y_proba = svm_model.predict_proba(X_test_scaled_df)[:, 1]


# Step 3: Metrics
accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred, pos_label=1)
mcc = matthews_corrcoef(y_test, y_pred)
auc = roc_auc_score(y_test, y_proba)
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
specificity = tn / (tn + fp)
f1 = f1_score(y_test_input, y_pred)

# Step 4: Cross-validated recall
cv_recall_scores = cross_val_score(
    svm_model, X_train, y_train, cv=5, scoring='recall'
)
cv_recall_mean = cv_recall_scores.mean()

# Step 5: Save results to CSV
header = [
    "Model_Title", "Accuracy", "Recall", "Specificity",
    "AUC", "MCC", "CV_Recall_Mean", "F1"
]

row = [
    model_title, accuracy, recall, specificity,
    auc, mcc, cv_recall_mean, f1
]


# Check if file exists to write header only once
file_exists = os.path.isfile(output_csv_path)

with open(output_csv_path, mode='a', newline='') as file:
    writer = csv.writer(file)
    if not file_exists:
        writer.writerow(header)
    writer.writerow(row)

print(f"✅ Results saved to {output_csv_path}")

# --- Step 5: Permutation feature importance
print(f"\n📊 Computing feature importances for: {model_title}")
result = permutation_importance(
    svm_model, X_test_scaled_df, y_test, n_repeats=10, random_state=42, scoring='recall'
)

importances = pd.Series(result.importances_mean, index=feature_names)

# 🔄 Transpose: model title as row, features as columns
importance_df = pd.DataFrame(
    importances.values.reshape(1, -1), columns=feature_names, index=[model_title]
)

if os.path.exists(feature_importance_csv):
    existing_df = pd.read_csv(feature_importance_csv, index_col=0)
    updated_df = pd.concat([existing_df, importance_df])
else:
    updated_df = importance_df

updated_df.to_csv(feature_importance_csv)

print(f"✅ Feature importances saved to {feature_importance_csv}")

In [ ]:
#SMOTE SVM
import csv
import os
import pandas as pd
from sklearn.metrics import (
    accuracy_score, recall_score, confusion_matrix,
    roc_auc_score, matthews_corrcoef
)
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.inspection import permutation_importance
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

# 🔁 Update this for each run
model_title = "SVM_scaled_SMOTE"  # Change name for each model you run
model = SVC(kernel='rbf', random_state=42, class_weight='balanced', probability=True)

# 🧠 Choose your input data (update manually depending on the variant)
X_train_input = X_train_resampled   # e.g., X_train, X_train_scaled, X_train_resampled
y_train_input = y_train_resampled   # y_train or y_train_resampled
X_test_input  = X_test_scaled       # e.g., X_test or X_test_scaled
y_test_input  = y_test              # usually y_test stays the same

# --- Step 1: Train model
model.fit(X_train_input, y_train_input)
y_pred = model.predict(X_test_input)
y_proba = model.predict_proba(X_test_input)[:, 1]

# --- Step 2: Evaluation metrics
accuracy = accuracy_score(y_test_input, y_pred)
recall = recall_score(y_test_input, y_pred)
mcc = matthews_corrcoef(y_test_input, y_pred)
auc = roc_auc_score(y_test_input, y_proba)
tn, fp, fn, tp = confusion_matrix(y_test_input, y_pred).ravel()
specificity = tn / (tn + fp)
f1 = f1_score(y_test_input, y_pred)


# --- Step 3: Cross-validated recall (no resampling, fair comparison)
pipeline = make_pipeline(StandardScaler(), model)
cv_recall_scores = cross_val_score(pipeline, X, y, cv=5, scoring='recall')
cv_recall_mean = cv_recall_scores.mean()

# --- Step 4: Save performance metrics
header = ["Model_Title", "Accuracy", "Recall", "Specificity", "AUC", "MCC", "CV_Recall_Mean", "F1"]
row = [model_title, accuracy, recall, specificity, auc, mcc, cv_recall_mean, f1]

file_exists = os.path.isfile(output_csv_path)
with open(output_csv_path, mode='a', newline='') as file:
    writer = csv.writer(file)
    if not file_exists:
        writer.writerow(header)
    writer.writerow(row)

print(f"✅ Metrics saved to {output_csv_path}")

# --- Step 5: Permutation feature importance
print(f"\n📊 Computing feature importances for: {model_title}")
result = permutation_importance(
    model, X_test_input, y_test_input, n_repeats=10, random_state=42, scoring='recall'
)
importances = pd.Series(result.importances_mean, index=feature_names)

# 🔄 Transpose: model title as row, features as columns
importance_df = pd.DataFrame(
    importances.values.reshape(1, -1), columns=feature_names, index=[model_title]
)

if os.path.exists(feature_importance_csv):
    existing_df = pd.read_csv(feature_importance_csv, index_col=0)
    updated_df = pd.concat([existing_df, importance_df])
else:
    updated_df = importance_df

updated_df.to_csv(feature_importance_csv)

print(f"✅ Feature importances saved to {feature_importance_csv}")


In [ ]:
# SVM + GridSearchCV

import csv
import os
import pandas as pd
from sklearn.metrics import (
    accuracy_score, recall_score, confusion_matrix,
    roc_auc_score, matthews_corrcoef
)
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.inspection import permutation_importance
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

# 🔁 Update this for each run
model_title = "SVM_gridsearch_original"  # Change this title for each run

# 🧠 Define your data inputs (can change this per run)
X_train_input = X_train
y_train_input = y_train
X_test_input  = X_test
y_test_input  = y_test

# Step 1: GridSearchCV setup
svm_base = SVC(probability=True, class_weight='balanced', random_state=42)
param_grid = {
    'C': [0.1, 1, 10],
    'gamma': ['scale', 0.01, 0.1],
    'kernel': ['rbf']
}

grid_search = GridSearchCV(
    estimator=svm_base,
    param_grid=param_grid,
    scoring='precision',  # Optimizing for precision on positive class
    cv=5,
    n_jobs=-1,
    verbose=2
)

# Step 2: Train and predict
grid_search.fit(X_train_input, y_train_input)
best_model = grid_search.best_estimator_
print(f"✅ Best params: {grid_search.best_params_}")

y_pred = best_model.predict(X_test_input)
y_proba = best_model.predict_proba(X_test_input)[:, 1]

# Step 3: Evaluation metrics
accuracy = accuracy_score(y_test_input, y_pred)
recall = recall_score(y_test_input, y_pred)
mcc = matthews_corrcoef(y_test_input, y_pred)
auc = roc_auc_score(y_test_input, y_proba)
tn, fp, fn, tp = confusion_matrix(y_test_input, y_pred).ravel()
specificity = tn / (tn + fp)
f1 = f1_score(y_test_input, y_pred)

# Step 4: Cross-validated recall (on full original data for fairness)
pipeline = make_pipeline(StandardScaler(), best_model)
cv_recall_scores = cross_val_score(pipeline, X, y, cv=5, scoring='recall')
cv_recall_mean = cv_recall_scores.mean()

# Step 5: Save evaluation metrics
header = [
    "Model_Title", "Accuracy", "Recall", "Specificity",
    "AUC", "MCC", "CV_Recall_Mean", "F1"
]

row = [
    model_title, accuracy, recall, specificity,
    auc, mcc, cv_recall_mean, f1
]

file_exists = os.path.isfile(output_csv_path)
with open(output_csv_path, mode='a', newline='') as file:
    writer = csv.writer(file)
    if not file_exists:
        writer.writerow(header)
    writer.writerow(row)

print(f"✅ Evaluation metrics saved to {output_csv_path}")

# Step 6: Permutation feature importance
print(f"\n📊 Computing permutation importances for: {model_title}")
result = permutation_importance(
    best_model, X_test_input, y_test_input, n_repeats=10,
    random_state=42, scoring='precision'
)

importances = pd.Series(result.importances_mean, index=feature_names)

importance_df = pd.DataFrame(
    importances.values.reshape(1, -1), columns=feature_names, index=[model_title]
)

# Append or create feature importance CSV
if os.path.exists(feature_importance_csv):
    existing_df = pd.read_csv(feature_importance_csv, index_col=0)
    updated_df = pd.concat([existing_df, importance_df])
else:
    updated_df = importance_df

updated_df.to_csv(feature_importance_csv)
print(f"✅ Feature importances saved to {feature_importance_csv}")


In [ ]:
# === SVM with Scaled Data and GridSearch ===

import csv
import os
import pandas as pd
from sklearn.svm import SVC
from sklearn.metrics import (
    accuracy_score, recall_score, confusion_matrix,
    roc_auc_score, matthews_corrcoef, f1_score
)
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.inspection import permutation_importance
from sklearn.preprocessing import StandardScaler

# --- Model title
model_title = "SVM_scaled_GridSearch"

# --- Prepare data
X_train_input = X_train
X_test_input = X_test
y_train_input = y_train
y_test_input = y_test

# --- Define pipeline with scaler and SVM
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('svm', SVC(kernel='rbf', class_weight='balanced', probability=True, random_state=42))
])

# --- Define parameter grid for SVM inside pipeline
param_grid = {
    'svm__C': [0.1, 1, 10],
    'svm__gamma': ['scale', 0.01, 0.1, 1]
}

grid = GridSearchCV(pipeline, param_grid, cv=3, scoring='recall', n_jobs=-1)
grid.fit(X_train_input, y_train_input)

# --- Best model
best_model = grid.best_estimator_
print(f"✅ Best model parameters: {grid.best_params_}")

# --- Predict
y_pred = best_model.predict(X_test_input)
y_proba = best_model.predict_proba(X_test_input)[:, 1]

# --- Metrics
accuracy = accuracy_score(y_test_input, y_pred)
recall = recall_score(y_test_input, y_pred)
mcc = matthews_corrcoef(y_test_input, y_pred)
auc = roc_auc_score(y_test_input, y_proba)
tn, fp, fn, tp = confusion_matrix(y_test_input, y_pred).ravel()
specificity = tn / (tn + fp)
f1 = f1_score(y_test_input, y_pred)

# --- Cross-validated recall (for comparison)
cv_recall_scores = cross_val_score(best_model, X, y, cv=5, scoring='recall')
cv_recall_mean = cv_recall_scores.mean()

# --- Save model evaluation
header = ["Model_Title", "Accuracy", "Recall", "Specificity", "AUC", "MCC", "CV_Recall_Mean", "F1"]
row = [model_title, accuracy, recall, specificity, auc, mcc, cv_recall_mean, f1]

file_exists = os.path.isfile(output_csv_path)
with open(output_csv_path, mode='a', newline='') as file:
    writer = csv.writer(file)
    if not file_exists:
        writer.writerow(header)
    writer.writerow(row)

print(f"✅ Evaluation metrics saved to {output_csv_path}")

# --- Permutation Feature Importance
print(f"\n📊 Computing permutation importances for: {model_title}")
result = permutation_importance(
    best_model, X_test_input, y_test_input, n_repeats=10, random_state=42, scoring='precision'
)

importances = pd.Series(result.importances_mean, index=feature_names)

# 🔄 Transpose and save
importance_df = pd.DataFrame(
    importances.values.reshape(1, -1),
    columns=feature_names,
    index=[model_title]
)

if os.path.exists(feature_importance_csv):
    existing_df = pd.read_csv(feature_importance_csv, index_col=0)
    updated_df = pd.concat([existing_df, importance_df])
else:
    updated_df = importance_df

updated_df.to_csv(feature_importance_csv)
print(f"✅ Feature importances saved to {feature_importance_csv}")


In [ ]:
#SVM smote+grid search 
import csv
import os
import pandas as pd
from imblearn.over_sampling import SMOTE
from sklearn.svm import SVC
from sklearn.metrics import (
    accuracy_score, recall_score, confusion_matrix,
    roc_auc_score, matthews_corrcoef, f1_score
)
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.inspection import permutation_importance

# --- Model title
model_title = "SVM_SMOTE_GridSearch"

# --- Apply SMOTE to training data (no scaling)
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# --- Define pipeline (NO SCALER)
pipeline = Pipeline([
    ('svm', SVC(kernel='rbf', class_weight='balanced', probability=True, random_state=42))
])

# --- Define parameter grid for SVM
param_grid = {
    'svm__C': [0.1, 1, 10],
    'svm__gamma': ['scale', 0.01, 0.1, 1]
}

# --- Grid Search
grid = GridSearchCV(pipeline, param_grid, cv=3, scoring='recall', n_jobs=-1)
grid.fit(X_train_resampled, y_train_resampled)

# --- Best model
best_model = grid.best_estimator_
print(f"✅ Best model parameters: {grid.best_params_}")

# --- Predict
y_pred = best_model.predict(X_test)
y_proba = best_model.predict_proba(X_test)[:, 1]

# --- Metrics
accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
mcc = matthews_corrcoef(y_test, y_pred)
auc = roc_auc_score(y_test, y_proba)
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
specificity = tn / (tn + fp)
f1 = f1_score(y_test, y_pred)

# --- Cross-validated recall (on original data for comparison)
cv_recall_scores = cross_val_score(best_model, X, y, cv=5, scoring='recall')
cv_recall_mean = cv_recall_scores.mean()

# --- Save model evaluation
header = ["Model_Title", "Accuracy", "Recall", "Specificity", "AUC", "MCC", "CV_Recall_Mean", "F1"]
row = [model_title, accuracy, recall, specificity, auc, mcc, cv_recall_mean, f1]

file_exists = os.path.isfile(output_csv_path)
with open(output_csv_path, mode='a', newline='') as file:
    writer = csv.writer(file)
    if not file_exists:
        writer.writerow(header)
    writer.writerow(row)

print(f"✅ Evaluation metrics saved to {output_csv_path}")

# --- Permutation Feature Importance
print(f"\n📊 Computing permutation importances for: {model_title}")
result = permutation_importance(
    best_model, X_test, y_test, n_repeats=10, random_state=42, scoring='precision'
)

importances = pd.Series(result.importances_mean, index=feature_names)

# 🔄 Transpose and save
importance_df = pd.DataFrame(
    importances.values.reshape(1, -1),
    columns=feature_names,
    index=[model_title]
)

if os.path.exists(feature_importance_csv):
    existing_df = pd.read_csv(feature_importance_csv, index_col=0)
    updated_df = pd.concat([existing_df, importance_df])
else:
    updated_df = importance_df

updated_df.to_csv(feature_importance_csv)
print(f"✅ Feature importances saved to {feature_importance_csv}")


In [ ]:
#SVM scaled+smote+gridseaerch 
import csv
import os
import pandas as pd
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline  # imblearn's pipeline, not sklearn's
from sklearn.svm import SVC
from sklearn.metrics import (
    accuracy_score, recall_score, confusion_matrix,
    roc_auc_score, matthews_corrcoef, f1_score
)
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.inspection import permutation_importance

# --- Model title
model_title = "SVM_SMOTE_Scaled_GridSearch"

# --- Define pipeline with scaling + SMOTE + SVM
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('smote', SMOTE(random_state=42)),
    ('svm', SVC(kernel='rbf', class_weight='balanced', probability=True, random_state=42))
])

# --- Define parameter grid
param_grid = {
    'svm__C': [0.1, 1, 10],
    'svm__gamma': ['scale', 0.01, 0.1, 1]
}

# --- Grid Search
grid = GridSearchCV(pipeline, param_grid, cv=3, scoring='recall', n_jobs=-1)
grid.fit(X_train, y_train)

# --- Best model
best_model = grid.best_estimator_
print(f"✅ Best model parameters: {grid.best_params_}")

# --- Predict
y_pred = best_model.predict(X_test)
y_proba = best_model.predict_proba(X_test)[:, 1]

# --- Metrics
accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
mcc = matthews_corrcoef(y_test, y_pred)
auc = roc_auc_score(y_test, y_proba)
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
specificity = tn / (tn + fp)
f1 = f1_score(y_test, y_pred)

# --- Cross-validated recall on original data
cv_recall_scores = cross_val_score(best_model, X, y, cv=5, scoring='recall')
cv_recall_mean = cv_recall_scores.mean()

# --- Save model evaluation
header = ["Model_Title", "Accuracy", "Recall", "Specificity", "AUC", "MCC", "CV_Recall_Mean", "F1"]
row = [model_title, accuracy, recall, specificity, auc, mcc, cv_recall_mean, f1]

file_exists = os.path.isfile(output_csv_path)
with open(output_csv_path, mode='a', newline='') as file:
    writer = csv.writer(file)
    if not file_exists:
        writer.writerow(header)
    writer.writerow(row)

print(f"✅ Evaluation metrics saved to {output_csv_path}")

# --- Permutation Feature Importance
print(f"\n📊 Computing permutation importances for: {model_title}")
result = permutation_importance(
    best_model, X_test, y_test, n_repeats=10, random_state=42, scoring='precision'
)

importances = pd.Series(result.importances_mean, index=feature_names)

# 🔄 Transpose and save
importance_df = pd.DataFrame(
    importances.values.reshape(1, -1),
    columns=feature_names,
    index=[model_title]
)

if os.path.exists(feature_importance_csv):
    existing_df = pd.read_csv(feature_importance_csv, index_col=0)
    updated_df = pd.concat([existing_df, importance_df])
else:
    updated_df = importance_df

updated_df.to_csv(feature_importance_csv)
print(f"✅ Feature importances saved to {feature_importance_csv}")


In [ ]:
#Decision Tree

In [ ]:
# === Decision Tree Classification ===

import csv
import os
import pandas as pd
from sklearn.metrics import (
    accuracy_score, recall_score, confusion_matrix,
    roc_auc_score, matthews_corrcoef, f1_score
)
from sklearn.model_selection import cross_val_score
from sklearn.inspection import permutation_importance
from sklearn.tree import DecisionTreeClassifier


# === Set model title ===
model_title = "DecisionTree_original"  # 🔁 Change this for each run

# === Train the Decision Tree ===
model = DecisionTreeClassifier(random_state=42, class_weight='balanced')
model.fit(X_train, y_train)

# === Predictions ===
y_pred = model.predict(X_test)
y_proba = model.predict_proba(X_test)[:, 1]

# === Metrics ===
accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred, pos_label=1)
mcc = matthews_corrcoef(y_test, y_pred)
auc = roc_auc_score(y_test, y_proba)
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
specificity = tn / (tn + fp)
f1 = f1_score(y_test, y_pred)

# === Cross-validated Recall ===
cv_recall_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='recall')
cv_recall_mean = cv_recall_scores.mean()

# === Save metrics to CSV ===
header = [
    "Model_Title", "Accuracy", "Recall", "Specificity",
    "AUC", "MCC", "CV_Recall_Mean", "F1"
]

row = [
    model_title, accuracy, recall, specificity,
    auc, mcc, cv_recall_mean, f1
]

file_exists = os.path.isfile(output_csv_path)

with open(output_csv_path, mode='a', newline='') as file:
    writer = csv.writer(file)
    if not file_exists:
        writer.writerow(header)
    writer.writerow(row)

print(f"✅ Results saved to {output_csv_path}")

# === Permutation Feature Importance ===
print(f"\n📊 Computing feature importances for: {model_title}")
result = permutation_importance(
    model, X_test, y_test, n_repeats=10, random_state=42, scoring='recall'
)
importances = pd.Series(result.importances_mean, index=feature_names)

# === Save feature importances ===
importance_df = pd.DataFrame(
    importances.values.reshape(1, -1), columns=feature_names, index=[model_title]
)

if os.path.exists(feature_importance_csv):
    existing_df = pd.read_csv(feature_importance_csv, index_col=0)
    updated_df = pd.concat([existing_df, importance_df])
else:
    updated_df = importance_df

updated_df.to_csv(feature_importance_csv)

print(f"✅ Feature importances saved to {feature_importance_csv}")


In [ ]:
#Decision tree scaled
import csv
import os
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score, recall_score, confusion_matrix,
    roc_auc_score, matthews_corrcoef, f1_score
)
from sklearn.model_selection import cross_val_score
from sklearn.inspection import permutation_importance
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline

# === Set model title ===
model_title = "DecisionTree_Scaled_NoSMOTE_NoGridSearch"

# === Define pipeline with scaler and Decision Tree ===
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', DecisionTreeClassifier(random_state=42, class_weight='balanced'))
])

# === Train model ===
pipeline.fit(X_train, y_train)

# === Predictions ===
y_pred = pipeline.predict(X_test)
y_proba = pipeline.predict_proba(X_test)[:, 1]

# === Metrics ===
accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred, pos_label=1)
mcc = matthews_corrcoef(y_test, y_pred)
auc = roc_auc_score(y_test, y_proba)
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
specificity = tn / (tn + fp)
f1 = f1_score(y_test, y_pred)

# === Cross-validated Recall ===
cv_recall_scores = cross_val_score(pipeline, X_train, y_train, cv=5, scoring='recall')
cv_recall_mean = cv_recall_scores.mean()

# === Save metrics to CSV ===
header = [
    "Model_Title", "Accuracy", "Recall", "Specificity",
    "AUC", "MCC", "CV_Recall_Mean", "F1"
]

row = [
    model_title, accuracy, recall, specificity,
    auc, mcc, cv_recall_mean, f1
]

file_exists = os.path.isfile(output_csv_path)

with open(output_csv_path, mode='a', newline='') as file:
    writer = csv.writer(file)
    if not file_exists:
        writer.writerow(header)
    writer.writerow(row)

print(f"✅ Results saved to {output_csv_path}")

# === Permutation Feature Importance ===
print(f"\n📊 Computing feature importances for: {model_title}")
result = permutation_importance(
    pipeline, X_test, y_test, n_repeats=10, random_state=42, scoring='recall'
)
importances = pd.Series(result.importances_mean, index=feature_names)

# === Save feature importances ===
importance_df = pd.DataFrame(
    importances.values.reshape(1, -1), columns=feature_names, index=[model_title]
)

if os.path.exists(feature_importance_csv):
    existing_df = pd.read_csv(feature_importance_csv, index_col=0)
    updated_df = pd.concat([existing_df, importance_df])
else:
    updated_df = importance_df

updated_df.to_csv(feature_importance_csv)
print(f"✅ Feature importances saved to {feature_importance_csv}")


In [ ]:
#Decision tree smote
import csv
import os
import pandas as pd
from imblearn.over_sampling import SMOTE
from sklearn.metrics import (
    accuracy_score, recall_score, confusion_matrix,
    roc_auc_score, matthews_corrcoef, f1_score
)
from sklearn.model_selection import cross_val_score
from sklearn.inspection import permutation_importance
from sklearn.tree import DecisionTreeClassifier

# === Set model title ===
model_title = "DecisionTree_SMOTE_NoGridSearch_NoScaling"

# === Apply SMOTE to training data ===
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# === Define and train model ===
model = DecisionTreeClassifier(random_state=42, class_weight='balanced')
model.fit(X_train_resampled, y_train_resampled)

# === Predictions ===
y_pred = model.predict(X_test)
y_proba = model.predict_proba(X_test)[:, 1]

# === Metrics ===
accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred, pos_label=1)
mcc = matthews_corrcoef(y_test, y_pred)
auc = roc_auc_score(y_test, y_proba)
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
specificity = tn / (tn + fp)
f1 = f1_score(y_test, y_pred)

# === Cross-validated Recall (using resampled training data) ===
cv_recall_scores = cross_val_score(model, X_train_resampled, y_train_resampled, cv=5, scoring='recall')
cv_recall_mean = cv_recall_scores.mean()

# === Save metrics to CSV ===
header = [
    "Model_Title", "Accuracy", "Recall", "Specificity",
    "AUC", "MCC", "CV_Recall_Mean", "F1"
]

row = [
    model_title, accuracy, recall, specificity,
    auc, mcc, cv_recall_mean, f1
]

file_exists = os.path.isfile(output_csv_path)

with open(output_csv_path, mode='a', newline='') as file:
    writer = csv.writer(file)
    if not file_exists:
        writer.writerow(header)
    writer.writerow(row)

print(f"✅ Results saved to {output_csv_path}")

# === Permutation Feature Importance ===
print(f"\n📊 Computing feature importances for: {model_title}")
result = permutation_importance(
    model, X_test, y_test, n_repeats=10, random_state=42, scoring='recall'
)
importances = pd.Series(result.importances_mean, index=feature_names)

# === Save feature importances ===
importance_df = pd.DataFrame(
    importances.values.reshape(1, -1), columns=feature_names, index=[model_title]
)

if os.path.exists(feature_importance_csv):
    existing_df = pd.read_csv(feature_importance_csv, index_col=0)
    updated_df = pd.concat([existing_df, importance_df])
else:
    updated_df = importance_df

updated_df.to_csv(feature_importance_csv)
print(f"✅ Feature importances saved to {feature_importance_csv}")


In [ ]:
#Decisioon Tree grid search 
import csv
import os
import pandas as pd
from sklearn.metrics import (
    accuracy_score, recall_score, confusion_matrix,
    roc_auc_score, matthews_corrcoef, f1_score
)
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.inspection import permutation_importance
from sklearn.tree import DecisionTreeClassifier

# === Set model title ===
model_title = "DecisionTree_GridSearch_NoSMOTE_NoScaling"

# === Define classifier ===
clf = DecisionTreeClassifier(random_state=42, class_weight='balanced')

# === Define parameter grid for GridSearch ===
param_grid = {
    'max_depth': [None, 5, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# === Run GridSearchCV ===
grid = GridSearchCV(clf, param_grid, cv=3, scoring='recall', n_jobs=-1, verbose=1)
grid.fit(X_train, y_train)

# === Best model ===
best_model = grid.best_estimator_
print(f"✅ Best model parameters: {grid.best_params_}")

# === Predictions ===
y_pred = best_model.predict(X_test)
y_proba = best_model.predict_proba(X_test)[:, 1]

# === Metrics ===
accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred, pos_label=1)
mcc = matthews_corrcoef(y_test, y_pred)
auc = roc_auc_score(y_test, y_proba)
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
specificity = tn / (tn + fp)
f1 = f1_score(y_test, y_pred)

# === Cross-validated Recall ===
cv_recall_scores = cross_val_score(best_model, X_train, y_train, cv=5, scoring='recall')
cv_recall_mean = cv_recall_scores.mean()

# === Save metrics to CSV ===
header = [
    "Model_Title", "Accuracy", "Recall", "Specificity",
    "AUC", "MCC", "CV_Recall_Mean", "F1"
]

row = [
    model_title, accuracy, recall, specificity,
    auc, mcc, cv_recall_mean, f1
]

file_exists = os.path.isfile(output_csv_path)

with open(output_csv_path, mode='a', newline='') as file:
    writer = csv.writer(file)
    if not file_exists:
        writer.writerow(header)
    writer.writerow(row)

print(f"✅ Results saved to {output_csv_path}")

# === Permutation Feature Importance ===
print(f"\n📊 Computing feature importances for: {model_title}")
result = permutation_importance(
    best_model, X_test, y_test, n_repeats=10, random_state=42, scoring='recall'
)
importances = pd.Series(result.importances_mean, index=feature_names)

# === Save feature importances ===
importance_df = pd.DataFrame(
    importances.values.reshape(1, -1), columns=feature_names, index=[model_title]
)

if os.path.exists(feature_importance_csv):
    existing_df = pd.read_csv(feature_importance_csv, index_col=0)
    updated_df = pd.concat([existing_df, importance_df])
else:
    updated_df = importance_df

updated_df.to_csv(feature_importance_csv)
print(f"✅ Feature importances saved to {feature_importance_csv}")


In [ ]:
#Decision tree scaled +grid search 
import csv
import os
import pandas as pd
from sklearn.metrics import (
    accuracy_score, recall_score, confusion_matrix,
    roc_auc_score, matthews_corrcoef, f1_score
)
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.inspection import permutation_importance
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline  # sklearn pipeline

# === Set model title ===
model_title = "DecisionTree_Scaled_GridSearch"

# === Define pipeline: Scaling -> Decision Tree (No SMOTE)
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', DecisionTreeClassifier(random_state=42, class_weight='balanced'))
])

# === Define parameter grid for GridSearch ===
param_grid = {
    'clf__max_depth': [None, 5, 10, 20],
    'clf__min_samples_split': [2, 5, 10],
    'clf__min_samples_leaf': [1, 2, 4]
}

# === Run GridSearchCV ===
grid = GridSearchCV(pipeline, param_grid, cv=3, scoring='recall', n_jobs=-1)
grid.fit(X_train, y_train)

# === Best model ===
best_model = grid.best_estimator_
print(f"✅ Best model parameters: {grid.best_params_}")

# === Predictions ===
y_pred = best_model.predict(X_test)
y_proba = best_model.predict_proba(X_test)[:, 1]

# === Metrics ===
accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred, pos_label=1)
mcc = matthews_corrcoef(y_test, y_pred)
auc = roc_auc_score(y_test, y_proba)
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
specificity = tn / (tn + fp)
f1 = f1_score(y_test, y_pred)

# === Cross-validated Recall ===
cv_recall_scores = cross_val_score(best_model, X_train, y_train, cv=5, scoring='recall')
cv_recall_mean = cv_recall_scores.mean()

# === Save metrics to CSV ===
header = [
    "Model_Title", "Accuracy", "Recall", "Specificity",
    "AUC", "MCC", "CV_Recall_Mean", "F1"
]

row = [
    model_title, accuracy, recall, specificity,
    auc, mcc, cv_recall_mean, f1
]

file_exists = os.path.isfile(output_csv_path)

with open(output_csv_path, mode='a', newline='') as file:
    writer = csv.writer(file)
    if not file_exists:
        writer.writerow(header)
    writer.writerow(row)

print(f"✅ Results saved to {output_csv_path}")

# === Permutation Feature Importance ===
print(f"\n📊 Computing feature importances for: {model_title}")
result = permutation_importance(
    best_model, X_test, y_test, n_repeats=10, random_state=42, scoring='recall'
)
importances = pd.Series(result.importances_mean, index=feature_names)

# === Save feature importances ===
importance_df = pd.DataFrame(
    importances.values.reshape(1, -1), columns=feature_names, index=[model_title]
)

if os.path.exists(feature_importance_csv):
    existing_df = pd.read_csv(feature_importance_csv, index_col=0)
    updated_df = pd.concat([existing_df, importance_df])
else:
    updated_df = importance_df

updated_df.to_csv(feature_importance_csv)
print(f"✅ Feature importances saved to {feature_importance_csv}")


In [ ]:
#Decision tree smote +grid search 
import csv
import os
import pandas as pd
from sklearn.metrics import (
    accuracy_score, recall_score, confusion_matrix,
    roc_auc_score, matthews_corrcoef, f1_score
)
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.inspection import permutation_importance
from sklearn.tree import DecisionTreeClassifier
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline  # Use imblearn pipeline

# === Set model title ===
model_title = "DecisionTree_SMOTE_GridSearch"

# === Define pipeline: SMOTE -> Decision Tree (NO SCALING)
pipeline = ImbPipeline([
    ('smote', SMOTE(random_state=42)),
    ('clf', DecisionTreeClassifier(random_state=42, class_weight='balanced'))
])

# === Define parameter grid for GridSearch ===
param_grid = {
    'clf__max_depth': [None, 5, 10, 20],
    'clf__min_samples_split': [2, 5, 10],
    'clf__min_samples_leaf': [1, 2, 4]
}

# === Run GridSearchCV ===
grid = GridSearchCV(pipeline, param_grid, cv=3, scoring='recall', n_jobs=-1)
grid.fit(X_train, y_train)

# === Best model ===
best_model = grid.best_estimator_
print(f"✅ Best model parameters: {grid.best_params_}")

# === Predictions ===
y_pred = best_model.predict(X_test)
y_proba = best_model.predict_proba(X_test)[:, 1]

# === Metrics ===
accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred, pos_label=1)
mcc = matthews_corrcoef(y_test, y_pred)
auc = roc_auc_score(y_test, y_proba)
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
specificity = tn / (tn + fp)
f1 = f1_score(y_test, y_pred)

# === Cross-validated Recall ===
cv_recall_scores = cross_val_score(best_model, X_train, y_train, cv=5, scoring='recall')
cv_recall_mean = cv_recall_scores.mean()

# === Save metrics to CSV ===
header = [
    "Model_Title", "Accuracy", "Recall", "Specificity",
    "AUC", "MCC", "CV_Recall_Mean", "F1"
]

row = [
    model_title, accuracy, recall, specificity,
    auc, mcc, cv_recall_mean, f1
]

file_exists = os.path.isfile(output_csv_path)

with open(output_csv_path, mode='a', newline='') as file:
    writer = csv.writer(file)
    if not file_exists:
        writer.writerow(header)
    writer.writerow(row)

print(f"✅ Results saved to {output_csv_path}")

# === Permutation Feature Importance ===
print(f"\n📊 Computing feature importances for: {model_title}")
result = permutation_importance(
    best_model, X_test, y_test, n_repeats=10, random_state=42, scoring='recall'
)
importances = pd.Series(result.importances_mean, index=feature_names)

# === Save feature importances ===
importance_df = pd.DataFrame(
    importances.values.reshape(1, -1), columns=feature_names, index=[model_title]
)

if os.path.exists(feature_importance_csv):
    existing_df = pd.read_csv(feature_importance_csv, index_col=0)
    updated_df = pd.concat([existing_df, importance_df])
else:
    updated_df = importance_df

updated_df.to_csv(feature_importance_csv)
print(f"✅ Feature importances saved to {feature_importance_csv}")


In [ ]:
#Decision tree scaled +smote +gridsearch 
import csv
import os
import pandas as pd
from sklearn.metrics import (
    accuracy_score, recall_score, confusion_matrix,
    roc_auc_score, matthews_corrcoef, f1_score
)
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.inspection import permutation_importance
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline  # imblearn's pipeline

# === Set model title ===
model_title = "DecisionTree_Scaled_SMOTE_GridSearch"

# === Define pipeline: Scaling -> SMOTE -> Decision Tree ===
pipeline = ImbPipeline([
    ('scaler', StandardScaler()),
    ('smote', SMOTE(random_state=42)),
    ('clf', DecisionTreeClassifier(random_state=42, class_weight='balanced'))
])

# === Define parameter grid for GridSearch ===
param_grid = {
    'clf__max_depth': [None, 5, 10, 20],
    'clf__min_samples_split': [2, 5, 10],
    'clf__min_samples_leaf': [1, 2, 4]
}

# === Run GridSearchCV ===
grid = GridSearchCV(pipeline, param_grid, cv=3, scoring='recall', n_jobs=-1)
grid.fit(X_train, y_train)

# === Best model ===
best_model = grid.best_estimator_
print(f"✅ Best model parameters: {grid.best_params_}")

# === Predictions ===
y_pred = best_model.predict(X_test)
y_proba = best_model.predict_proba(X_test)[:, 1]

# === Metrics ===
accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred, pos_label=1)
mcc = matthews_corrcoef(y_test, y_pred)
auc = roc_auc_score(y_test, y_proba)
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
specificity = tn / (tn + fp)
f1 = f1_score(y_test, y_pred)

# === Cross-validated Recall ===
cv_recall_scores = cross_val_score(best_model, X_train, y_train, cv=5, scoring='recall')
cv_recall_mean = cv_recall_scores.mean()

# === Save metrics to CSV ===
header = [
    "Model_Title", "Accuracy", "Recall", "Specificity",
    "AUC", "MCC", "CV_Recall_Mean", "F1"
]

row = [
    model_title, accuracy, recall, specificity,
    auc, mcc, cv_recall_mean, f1
]

file_exists = os.path.isfile(output_csv_path)

with open(output_csv_path, mode='a', newline='') as file:
    writer = csv.writer(file)
    if not file_exists:
        writer.writerow(header)
    writer.writerow(row)

print(f"✅ Results saved to {output_csv_path}")

# === Permutation Feature Importance ===
print(f"\n📊 Computing feature importances for: {model_title}")
result = permutation_importance(
    best_model, X_test, y_test, n_repeats=10, random_state=42, scoring='recall'
)
importances = pd.Series(result.importances_mean, index=feature_names)

# === Save feature importances ===
importance_df = pd.DataFrame(
    importances.values.reshape(1, -1), columns=feature_names, index=[model_title]
)

if os.path.exists(feature_importance_csv):
    existing_df = pd.read_csv(feature_importance_csv, index_col=0)
    updated_df = pd.concat([existing_df, importance_df])
else:
    updated_df = importance_df

updated_df.to_csv(feature_importance_csv)
print(f"✅ Feature importances saved to {feature_importance_csv}")


In [ ]:
#Random Forest

In [ ]:
#RF standard
import csv
import os
import pandas as pd
from sklearn.metrics import (
    accuracy_score, recall_score, confusion_matrix,
    roc_auc_score, matthews_corrcoef, f1_score
)
from sklearn.model_selection import cross_val_score
from sklearn.inspection import permutation_importance
from sklearn.ensemble import RandomForestClassifier  # <-- Changed here


# === Set model title ===
model_title = "RandomForest_original"  # Changed model title accordingly

# === Train the Random Forest ===
model = RandomForestClassifier(random_state=42, class_weight='balanced')
model.fit(X_train, y_train)

# === Predictions ===
y_pred = model.predict(X_test)
y_proba = model.predict_proba(X_test)[:, 1]

# === Metrics ===
accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred, pos_label=1)
mcc = matthews_corrcoef(y_test, y_pred)
auc = roc_auc_score(y_test, y_proba)
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
specificity = tn / (tn + fp)
f1 = f1_score(y_test, y_pred)

# === Cross-validated Recall ===
cv_recall_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='recall')
cv_recall_mean = cv_recall_scores.mean()

# === Save metrics to CSV ===
header = [
    "Model_Title", "Accuracy", "Recall", "Specificity",
    "AUC", "MCC", "CV_Recall_Mean", "F1"
]

row = [
    model_title, accuracy, recall, specificity,
    auc, mcc, cv_recall_mean, f1
]

file_exists = os.path.isfile(output_csv_path)

with open(output_csv_path, mode='a', newline='') as file:
    writer = csv.writer(file)
    if not file_exists:
        writer.writerow(header)
    writer.writerow(row)

print(f"✅ Results saved to {output_csv_path}")

# === Permutation Feature Importance ===
print(f"\n📊 Computing feature importances for: {model_title}")
result = permutation_importance(
    model, X_test, y_test, n_repeats=10, random_state=42, scoring='recall'
)
importances = pd.Series(result.importances_mean, index=feature_names)

# === Save feature importances ===
importance_df = pd.DataFrame(
    importances.values.reshape(1, -1), columns=feature_names, index=[model_title]
)

if os.path.exists(feature_importance_csv):
    existing_df = pd.read_csv(feature_importance_csv, index_col=0)
    updated_df = pd.concat([existing_df, importance_df])
else:
    updated_df = importance_df

updated_df.to_csv(feature_importance_csv)

print(f"✅ Feature importances saved to {feature_importance_csv}")


In [ ]:
#RF scaled
import csv
import os
import pandas as pd
from sklearn.metrics import (
    accuracy_score, recall_score, confusion_matrix,
    roc_auc_score, matthews_corrcoef, f1_score
)
from sklearn.model_selection import cross_val_score
from sklearn.inspection import permutation_importance
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# === Set model title ===
model_title = "RandomForest_scaled"

# === Create pipeline with scaler and RF ===
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('rf', RandomForestClassifier(random_state=42, class_weight='balanced'))
])

# === Train model using pipeline ===
pipeline.fit(X_train, y_train)

# === Predictions ===
y_pred = pipeline.predict(X_test)
y_proba = pipeline.predict_proba(X_test)[:, 1]

# === Metrics ===
accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred, pos_label=1)
mcc = matthews_corrcoef(y_test, y_pred)
auc = roc_auc_score(y_test, y_proba)
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
specificity = tn / (tn + fp)
f1 = f1_score(y_test, y_pred)

# === Cross-validated Recall ===
cv_recall_scores = cross_val_score(pipeline, X_train, y_train, cv=5, scoring='recall')
cv_recall_mean = cv_recall_scores.mean()

# === Save metrics to CSV ===
header = [
    "Model_Title", "Accuracy", "Recall", "Specificity",
    "AUC", "MCC", "CV_Recall_Mean", "F1"
]

row = [
    model_title, accuracy, recall, specificity,
    auc, mcc, cv_recall_mean, f1
]

file_exists = os.path.isfile(output_csv_path)

with open(output_csv_path, mode='a', newline='') as file:
    writer = csv.writer(file)
    if not file_exists:
        writer.writerow(header)
    writer.writerow(row)

print(f"✅ Results saved to {output_csv_path}")

# === Permutation Feature Importance ===
print(f"\n📊 Computing feature importances for: {model_title}")
result = permutation_importance(
    pipeline, X_test, y_test, n_repeats=10, random_state=42, scoring='recall'
)
importances = pd.Series(result.importances_mean, index=feature_names)

# === Save feature importances ===
importance_df = pd.DataFrame(
    importances.values.reshape(1, -1), columns=feature_names, index=[model_title]
)

if os.path.exists(feature_importance_csv):
    existing_df = pd.read_csv(feature_importance_csv, index_col=0)
    updated_df = pd.concat([existing_df, importance_df])
else:
    updated_df = importance_df

updated_df.to_csv(feature_importance_csv)

print(f"✅ Feature importances saved to {feature_importance_csv}")


In [ ]:
#RF Smote
import csv
import os
import pandas as pd
from sklearn.metrics import (
    accuracy_score, recall_score, confusion_matrix,
    roc_auc_score, matthews_corrcoef, f1_score
)
from sklearn.model_selection import cross_val_score
from sklearn.inspection import permutation_importance
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline  # imblearn's pipeline to handle SMOTE

# === Set model title ===
model_title = "RandomForest_SMOTE"

# === Create pipeline with SMOTE and Random Forest ===
pipeline = Pipeline([
    ('smote', SMOTE(random_state=42)),
    ('rf', RandomForestClassifier(random_state=42, class_weight='balanced'))
])

# === Train model using pipeline ===
pipeline.fit(X_train, y_train)

# === Predictions ===
y_pred = pipeline.predict(X_test)
y_proba = pipeline.predict_proba(X_test)[:, 1]

# === Metrics ===
accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred, pos_label=1)
mcc = matthews_corrcoef(y_test, y_pred)
auc = roc_auc_score(y_test, y_proba)
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
specificity = tn / (tn + fp)
f1 = f1_score(y_test, y_pred)

# === Cross-validated Recall ===
cv_recall_scores = cross_val_score(pipeline, X_train, y_train, cv=5, scoring='recall')
cv_recall_mean = cv_recall_scores.mean()

# === Save metrics to CSV ===
header = [
    "Model_Title", "Accuracy", "Recall", "Specificity",
    "AUC", "MCC", "CV_Recall_Mean", "F1"
]

row = [
    model_title, accuracy, recall, specificity,
    auc, mcc, cv_recall_mean, f1
]

file_exists = os.path.isfile(output_csv_path)

with open(output_csv_path, mode='a', newline='') as file:
    writer = csv.writer(file)
    if not file_exists:
        writer.writerow(header)
    writer.writerow(row)

print(f"✅ Results saved to {output_csv_path}")

# === Permutation Feature Importance ===
print(f"\n📊 Computing feature importances for: {model_title}")
result = permutation_importance(
    pipeline, X_test, y_test, n_repeats=10, random_state=42, scoring='recall'
)
importances = pd.Series(result.importances_mean, index=feature_names)

# === Save feature importances ===
importance_df = pd.DataFrame(
    importances.values.reshape(1, -1), columns=feature_names, index=[model_title]
)

if os.path.exists(feature_importance_csv):
    existing_df = pd.read_csv(feature_importance_csv, index_col=0)
    updated_df = pd.concat([existing_df, importance_df])
else:
    updated_df = importance_df

updated_df.to_csv(feature_importance_csv)

print(f"✅ Feature importances saved to {feature_importance_csv}")


In [ ]:
#RF grid search 
import csv
import os
import pandas as pd
from sklearn.metrics import (
    accuracy_score, recall_score, confusion_matrix,
    roc_auc_score, matthews_corrcoef, f1_score
)
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.inspection import permutation_importance
from sklearn.ensemble import RandomForestClassifier

# === Set model title ===
model_title = "RandomForest_GridSearch"

# === Define Random Forest model ===
rf = RandomForestClassifier(random_state=42, class_weight='balanced')

# === Define parameter grid for GridSearch ===
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

# === GridSearchCV ===
grid = GridSearchCV(rf, param_grid, cv=3, scoring='recall', n_jobs=-1)
grid.fit(X_train, y_train)

# === Best model ===
best_model = grid.best_estimator_
print(f"✅ Best model parameters: {grid.best_params_}")

# === Predictions ===
y_pred = best_model.predict(X_test)
y_proba = best_model.predict_proba(X_test)[:, 1]

# === Metrics ===
accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred, pos_label=1)
mcc = matthews_corrcoef(y_test, y_pred)
auc = roc_auc_score(y_test, y_proba)
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
specificity = tn / (tn + fp)
f1 = f1_score(y_test, y_pred)

# === Cross-validated Recall ===
cv_recall_scores = cross_val_score(best_model, X_train, y_train, cv=5, scoring='recall')
cv_recall_mean = cv_recall_scores.mean()

# === Save metrics to CSV ===
header = [
    "Model_Title", "Accuracy", "Recall", "Specificity",
    "AUC", "MCC", "CV_Recall_Mean", "F1"
]

row = [
    model_title, accuracy, recall, specificity,
    auc, mcc, cv_recall_mean, f1
]

file_exists = os.path.isfile(output_csv_path)

with open(output_csv_path, mode='a', newline='') as file:
    writer = csv.writer(file)
    if not file_exists:
        writer.writerow(header)
    writer.writerow(row)

print(f"✅ Results saved to {output_csv_path}")

# === Permutation Feature Importance ===
print(f"\n📊 Computing feature importances for: {model_title}")
result = permutation_importance(
    best_model, X_test, y_test, n_repeats=10, random_state=42, scoring='recall'
)
importances = pd.Series(result.importances_mean, index=feature_names)

# === Save feature importances ===
importance_df = pd.DataFrame(
    importances.values.reshape(1, -1), columns=feature_names, index=[model_title]
)

if os.path.exists(feature_importance_csv):
    existing_df = pd.read_csv(feature_importance_csv, index_col=0)
    updated_df = pd.concat([existing_df, importance_df])
else:
    updated_df = importance_df

updated_df.to_csv(feature_importance_csv)

print(f"✅ Feature importances saved to {feature_importance_csv}")


In [ ]:
#RF smote +grid search 
import csv
import os
import pandas as pd
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.metrics import (
    accuracy_score, recall_score, confusion_matrix,
    roc_auc_score, matthews_corrcoef, f1_score
)
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.inspection import permutation_importance
from sklearn.ensemble import RandomForestClassifier

# === Set model title ===
model_title = "RandomForest_SMOTE_GridSearch"

# === Define pipeline with SMOTE and Random Forest (no scaler) ===
pipeline = ImbPipeline([
    ('smote', SMOTE(random_state=42)),
    ('rf', RandomForestClassifier(random_state=42, class_weight='balanced'))
])

# === Define parameter grid for GridSearch ===
param_grid = {
    'rf__n_estimators': [50, 100, 200],
    'rf__max_depth': [None, 5, 10, 20],
    'rf__min_samples_split': [2, 5, 10],
    'rf__min_samples_leaf': [1, 2, 4]
}

# === GridSearchCV setup ===
grid = GridSearchCV(pipeline, param_grid, cv=3, scoring='recall', n_jobs=-1)
grid.fit(X_train, y_train)

# === Best model ===
best_model = grid.best_estimator_
print(f"✅ Best model parameters: {grid.best_params_}")

# === Predictions ===
y_pred = best_model.predict(X_test)
y_proba = best_model.predict_proba(X_test)[:, 1]

# === Metrics ===
accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
mcc = matthews_corrcoef(y_test, y_pred)
auc = roc_auc_score(y_test, y_proba)
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
specificity = tn / (tn + fp)
f1 = f1_score(y_test, y_pred)

# === Cross-validated Recall ===
cv_recall_scores = cross_val_score(best_model, X_train, y_train, cv=5, scoring='recall')
cv_recall_mean = cv_recall_scores.mean()

# === Save metrics to CSV ===
header = [
    "Model_Title", "Accuracy", "Recall", "Specificity",
    "AUC", "MCC", "CV_Recall_Mean", "F1"
]

row = [
    model_title, accuracy, recall, specificity,
    auc, mcc, cv_recall_mean, f1
]

file_exists = os.path.isfile(output_csv_path)

with open(output_csv_path, mode='a', newline='') as file:
    writer = csv.writer(file)
    if not file_exists:
        writer.writerow(header)
    writer.writerow(row)

print(f"✅ Results saved to {output_csv_path}")

# === Permutation Feature Importance ===
print(f"\n📊 Computing feature importances for: {model_title}")
result = permutation_importance(
    best_model, X_test, y_test, n_repeats=10, random_state=42, scoring='recall'
)
importances = pd.Series(result.importances_mean, index=feature_names)

# === Save feature importances ===
importance_df = pd.DataFrame(
    importances.values.reshape(1, -1), columns=feature_names, index=[model_title]
)

if os.path.exists(feature_importance_csv):
    existing_df = pd.read_csv(feature_importance_csv, index_col=0)
    updated_df = pd.concat([existing_df, importance_df])
else:
    updated_df = importance_df

updated_df.to_csv(feature_importance_csv)

print(f"✅ Feature importances saved to {feature_importance_csv}")


In [ ]:
#Neural Network

In [ ]:
#Neural Network Standard
import csv
import os
import pandas as pd
from sklearn.metrics import (
    accuracy_score, recall_score, confusion_matrix,
    roc_auc_score, matthews_corrcoef, f1_score
)
from sklearn.model_selection import cross_val_score
from sklearn.inspection import permutation_importance
from sklearn.neural_network import MLPClassifier

# === Set model title ===
model_title = "NeuralNetwork_original"  # Updated model title

# === Train the Neural Network ===
model = MLPClassifier(random_state=42, max_iter=1000)
model.fit(X_train, y_train)

# === Predictions ===
y_pred = model.predict(X_test)
# MLPClassifier has predict_proba only if 'solver' supports it (default 'adam' supports it)
y_proba = model.predict_proba(X_test)[:, 1]

# === Metrics ===
accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred, pos_label=1)
mcc = matthews_corrcoef(y_test, y_pred)
auc = roc_auc_score(y_test, y_proba)
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
specificity = tn / (tn + fp)
f1 = f1_score(y_test, y_pred)

# === Cross-validated Recall ===
cv_recall_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='recall')
cv_recall_mean = cv_recall_scores.mean()

# === Save metrics to CSV ===
header = [
    "Model_Title", "Accuracy", "Recall", "Specificity",
    "AUC", "MCC", "CV_Recall_Mean", "F1"
]

row = [
    model_title, accuracy, recall, specificity,
    auc, mcc, cv_recall_mean, f1
]

file_exists = os.path.isfile(output_csv_path)

with open(output_csv_path, mode='a', newline='') as file:
    writer = csv.writer(file)
    if not file_exists:
        writer.writerow(header)
    writer.writerow(row)

print(f"✅ Results saved to {output_csv_path}")

# === Permutation Feature Importance ===
print(f"\n📊 Computing feature importances for: {model_title}")
result = permutation_importance(
    model, X_test, y_test, n_repeats=10, random_state=42, scoring='recall'
)
importances = pd.Series(result.importances_mean, index=feature_names)

# === Save feature importances ===
importance_df = pd.DataFrame(
    importances.values.reshape(1, -1), columns=feature_names, index=[model_title]
)

if os.path.exists(feature_importance_csv):
    existing_df = pd.read_csv(feature_importance_csv, index_col=0)
    updated_df = pd.concat([existing_df, importance_df])
else:
    updated_df = importance_df

updated_df.to_csv(feature_importance_csv)

print(f"✅ Feature importances saved to {feature_importance_csv}")


In [ ]:
#NN with scaler
import csv
import os
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import (
    accuracy_score, recall_score, confusion_matrix,
    roc_auc_score, matthews_corrcoef, f1_score
)
from sklearn.model_selection import cross_val_score
from sklearn.inspection import permutation_importance

# === Set model title ===
model_title = "NeuralNetwork_Scaled"

# === Define pipeline: Scaling + Neural Network ===
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('mlp', MLPClassifier(random_state=42, max_iter=1000))
])

# === Fit model ===
pipeline.fit(X_train, y_train)

# === Predictions ===
y_pred = pipeline.predict(X_test)
y_proba = pipeline.predict_proba(X_test)[:, 1]

# === Metrics ===
accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
mcc = matthews_corrcoef(y_test, y_pred)
auc = roc_auc_score(y_test, y_proba)
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
specificity = tn / (tn + fp)
f1 = f1_score(y_test, y_pred)

# === Cross-validated Recall ===
cv_recall_scores = cross_val_score(pipeline, X_train, y_train, cv=5, scoring='recall')
cv_recall_mean = cv_recall_scores.mean()

# === Save metrics to CSV ===
header = [
    "Model_Title", "Accuracy", "Recall", "Specificity",
    "AUC", "MCC", "CV_Recall_Mean", "F1"
]

row = [
    model_title, accuracy, recall, specificity,
    auc, mcc, cv_recall_mean, f1
]

file_exists = os.path.isfile(output_csv_path)

with open(output_csv_path, mode='a', newline='') as file:
    writer = csv.writer(file)
    if not file_exists:
        writer.writerow(header)
    writer.writerow(row)

print(f"✅ Results saved to {output_csv_path}")

# === Permutation Feature Importance ===
print(f"\n📊 Computing feature importances for: {model_title}")
result = permutation_importance(
    pipeline, X_test, y_test, n_repeats=10, random_state=42, scoring='recall'
)
importances = pd.Series(result.importances_mean, index=feature_names)

# === Save feature importances ===
importance_df = pd.DataFrame(
    importances.values.reshape(1, -1), columns=feature_names, index=[model_title]
)

if os.path.exists(feature_importance_csv):
    existing_df = pd.read_csv(feature_importance_csv, index_col=0)
    updated_df = pd.concat([existing_df, importance_df])
else:
    updated_df = importance_df

updated_df.to_csv(feature_importance_csv)
print(f"✅ Feature importances saved to {feature_importance_csv}")


In [ ]:
#NN smote
import csv
import os
import pandas as pd
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import (
    accuracy_score, recall_score, confusion_matrix,
    roc_auc_score, matthews_corrcoef, f1_score
)
from sklearn.model_selection import cross_val_score
from sklearn.inspection import permutation_importance

# === Set model title ===
model_title = "NeuralNetwork_SMOTE"

# === Define pipeline: SMOTE + Neural Network (no scaler) ===
pipeline = ImbPipeline([
    ('smote', SMOTE(random_state=42)),
    ('mlp', MLPClassifier(random_state=42, max_iter=1000))
])

# === Fit model ===
pipeline.fit(X_train, y_train)

# === Predictions ===
y_pred = pipeline.predict(X_test)
y_proba = pipeline.predict_proba(X_test)[:, 1]

# === Metrics ===
accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
mcc = matthews_corrcoef(y_test, y_pred)
auc = roc_auc_score(y_test, y_proba)
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
specificity = tn / (tn + fp)
f1 = f1_score(y_test, y_pred)

# === Cross-validated Recall ===
cv_recall_scores = cross_val_score(pipeline, X_train, y_train, cv=5, scoring='recall')
cv_recall_mean = cv_recall_scores.mean()

# === Save metrics to CSV ===
header = [
    "Model_Title", "Accuracy", "Recall", "Specificity",
    "AUC", "MCC", "CV_Recall_Mean", "F1"
]

row = [
    model_title, accuracy, recall, specificity,
    auc, mcc, cv_recall_mean, f1
]

file_exists = os.path.isfile(output_csv_path)

with open(output_csv_path, mode='a', newline='') as file:
    writer = csv.writer(file)
    if not file_exists:
        writer.writerow(header)
    writer.writerow(row)

print(f"✅ Results saved to {output_csv_path}")

# === Permutation Feature Importance ===
print(f"\n📊 Computing feature importances for: {model_title}")
result = permutation_importance(
    pipeline, X_test, y_test, n_repeats=10, random_state=42, scoring='recall'
)
importances = pd.Series(result.importances_mean, index=feature_names)

# === Save feature importances ===
importance_df = pd.DataFrame(
    importances.values.reshape(1, -1), columns=feature_names, index=[model_title]
)

if os.path.exists(feature_importance_csv):
    existing_df = pd.read_csv(feature_importance_csv, index_col=0)
    updated_df = pd.concat([existing_df, importance_df])
else:
    updated_df = importance_df

updated_df.to_csv(feature_importance_csv)
print(f"✅ Feature importances saved to {feature_importance_csv}")


In [ ]:
#NN grid search  
import csv
import os
import pandas as pd
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import (
    accuracy_score, recall_score, confusion_matrix,
    roc_auc_score, matthews_corrcoef, f1_score
)
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.inspection import permutation_importance

# === Set model title ===
model_title = "NeuralNetwork_GridSearch"

# === Define model and parameter grid ===
param_grid = {
    'hidden_layer_sizes': [(50,), (100,), (50, 50)],
    'alpha': [0.0001, 0.001, 0.01],
    'solver': ['adam', 'lbfgs'],
    'learning_rate': ['constant', 'adaptive']
}

grid = GridSearchCV(
    estimator=MLPClassifier(random_state=42, max_iter=1000),
    param_grid=param_grid,
    scoring='recall',
    cv=3,
    n_jobs=-1
)

# === Fit model ===
grid.fit(X_train, y_train)

# === Best model ===
best_model = grid.best_estimator_
print(f"✅ Best model parameters: {grid.best_params_}")

# === Predictions ===
y_pred = best_model.predict(X_test)
y_proba = best_model.predict_proba(X_test)[:, 1]

# === Metrics ===
accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
mcc = matthews_corrcoef(y_test, y_pred)
auc = roc_auc_score(y_test, y_proba)
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
specificity = tn / (tn + fp)
f1 = f1_score(y_test, y_pred)

# === Cross-validated Recall ===
cv_recall_scores = cross_val_score(best_model, X_train, y_train, cv=5, scoring='recall')
cv_recall_mean = cv_recall_scores.mean()

# === Save metrics to CSV ===
header = [
    "Model_Title", "Accuracy", "Recall", "Specificity",
    "AUC", "MCC", "CV_Recall_Mean", "F1"
]

row = [
    model_title, accuracy, recall, specificity,
    auc, mcc, cv_recall_mean, f1
]

file_exists = os.path.isfile(output_csv_path)

with open(output_csv_path, mode='a', newline='') as file:
    writer = csv.writer(file)
    if not file_exists:
        writer.writerow(header)
    writer.writerow(row)

print(f"✅ Results saved to {output_csv_path}")

# === Permutation Feature Importance ===
print(f"\n📊 Computing feature importances for: {model_title}")
result = permutation_importance(
    best_model, X_test, y_test, n_repeats=10, random_state=42, scoring='recall'
)
importances = pd.Series(result.importances_mean, index=feature_names)

# === Save feature importances ===
importance_df = pd.DataFrame(
    importances.values.reshape(1, -1), columns=feature_names, index=[model_title]
)

if os.path.exists(feature_importance_csv):
    existing_df = pd.read_csv(feature_importance_csv, index_col=0)
    updated_df = pd.concat([existing_df, importance_df])
else:
    updated_df = importance_df

updated_df.to_csv(feature_importance_csv)
print(f"✅ Feature importances saved to {feature_importance_csv}")


In [ ]:
#NN grid search +smote
import csv
import os
import pandas as pd
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import SMOTE
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import (
    accuracy_score, recall_score, confusion_matrix,
    roc_auc_score, matthews_corrcoef, f1_score
)
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.inspection import permutation_importance

# === Set model title ===
model_title = "NeuralNetwork_SMOTE_GridSearch"

# === Define pipeline: SMOTE + Neural Network ===
pipeline = ImbPipeline([
    ('smote', SMOTE(random_state=42)),
    ('nn', MLPClassifier(random_state=42, max_iter=1000))
])

# === Define parameter grid for GridSearch ===
param_grid = {
    'nn__hidden_layer_sizes': [(50,), (100,), (50, 50)],
    'nn__alpha': [0.0001, 0.001, 0.01],
    'nn__solver': ['adam', 'lbfgs'],
    'nn__learning_rate': ['constant', 'adaptive']
}

grid = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    scoring='recall',
    cv=3,
    n_jobs=-1
)

# === Fit model ===
grid.fit(X_train, y_train)

# === Best model ===
best_model = grid.best_estimator_
print(f"✅ Best model parameters: {grid.best_params_}")

# === Predictions ===
y_pred = best_model.predict(X_test)
y_proba = best_model.predict_proba(X_test)[:, 1]

# === Metrics ===
accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
mcc = matthews_corrcoef(y_test, y_pred)
auc = roc_auc_score(y_test, y_proba)
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
specificity = tn / (tn + fp)
f1 = f1_score(y_test, y_pred)

# === Cross-validated Recall ===
cv_recall_scores = cross_val_score(best_model, X_train, y_train, cv=5, scoring='recall')
cv_recall_mean = cv_recall_scores.mean()

# === Save metrics to CSV ===
header = [
    "Model_Title", "Accuracy", "Recall", "Specificity",
    "AUC", "MCC", "CV_Recall_Mean", "F1"
]

row = [
    model_title, accuracy, recall, specificity,
    auc, mcc, cv_recall_mean, f1
]

file_exists = os.path.isfile(output_csv_path)

with open(output_csv_path, mode='a', newline='') as file:
    writer = csv.writer(file)
    if not file_exists:
        writer.writerow(header)
    writer.writerow(row)

print(f"✅ Results saved to {output_csv_path}")

# === Permutation Feature Importance ===
print(f"\n📊 Computing feature importances for: {model_title}")
result = permutation_importance(
    best_model, X_test, y_test, n_repeats=10, random_state=42, scoring='recall'
)
importances = pd.Series(result.importances_mean, index=feature_names)

# === Save feature importances ===
importance_df = pd.DataFrame(
    importances.values.reshape(1, -1), columns=feature_names, index=[model_title]
)

if os.path.exists(feature_importance_csv):
    existing_df = pd.read_csv(feature_importance_csv, index_col=0)
    updated_df = pd.concat([existing_df, importance_df])
else:
    updated_df = importance_df

updated_df.to_csv(feature_importance_csv)
print(f"✅ Feature importances saved to {feature_importance_csv}")


In [ ]:
#NN Scaled and grid search 
import csv
import os
import pandas as pd
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import (
    accuracy_score, recall_score, confusion_matrix,
    roc_auc_score, matthews_corrcoef, f1_score
)
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.inspection import permutation_importance
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# === Set model title ===
model_title = "NeuralNetwork_Scaled_GridSearch"

# === Define pipeline: Scaling -> Neural Network ===
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('nn', MLPClassifier(random_state=42, max_iter=1000))
])

# === Define parameter grid ===
param_grid = {
    'nn__hidden_layer_sizes': [(50,), (100,), (50, 50)],
    'nn__alpha': [0.0001, 0.001, 0.01],
    'nn__solver': ['adam', 'lbfgs'],
    'nn__learning_rate': ['constant', 'adaptive']
}

grid = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    scoring='recall',
    cv=3,
    n_jobs=-1
)

# === Fit model ===
grid.fit(X_train, y_train)

# === Best model ===
best_model = grid.best_estimator_
print(f"✅ Best model parameters: {grid.best_params_}")

# === Predictions ===
y_pred = best_model.predict(X_test)
y_proba = best_model.predict_proba(X_test)[:, 1]

# === Metrics ===
accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
mcc = matthews_corrcoef(y_test, y_pred)
auc = roc_auc_score(y_test, y_proba)
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
specificity = tn / (tn + fp)
f1 = f1_score(y_test, y_pred)

# === Cross-validated Recall ===
cv_recall_scores = cross_val_score(best_model, X_train, y_train, cv=5, scoring='recall')
cv_recall_mean = cv_recall_scores.mean()

# === Save metrics to CSV ===
header = [
    "Model_Title", "Accuracy", "Recall", "Specificity",
    "AUC", "MCC", "CV_Recall_Mean", "F1"
]

row = [
    model_title, accuracy, recall, specificity,
    auc, mcc, cv_recall_mean, f1
]

file_exists = os.path.isfile(output_csv_path)

with open(output_csv_path, mode='a', newline='') as file:
    writer = csv.writer(file)
    if not file_exists:
        writer.writerow(header)
    writer.writerow(row)

print(f"✅ Results saved to {output_csv_path}")

# === Permutation Feature Importance ===
print(f"\n📊 Computing feature importances for: {model_title}")
result = permutation_importance(
    best_model, X_test, y_test, n_repeats=10, random_state=42, scoring='recall'
)
importances = pd.Series(result.importances_mean, index=feature_names)

# === Save feature importances ===
importance_df = pd.DataFrame(
    importances.values.reshape(1, -1), columns=feature_names, index=[model_title]
)

if os.path.exists(feature_importance_csv):
    existing_df = pd.read_csv(feature_importance_csv, index_col=0)
    updated_df = pd.concat([existing_df, importance_df])
else:
    updated_df = importance_df

updated_df.to_csv(feature_importance_csv)
print(f"✅ Feature importances saved to {feature_importance_csv}")


In [ ]:
#NN scaled +smote+gridsearch
import csv
import os
import pandas as pd
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import SMOTE
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score, recall_score, confusion_matrix,
    roc_auc_score, matthews_corrcoef, f1_score
)
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.inspection import permutation_importance

# === Set model title ===
model_title = "NeuralNetwork_Scaled_SMOTE_GridSearch"

# === Define pipeline: Scaling -> SMOTE -> Neural Network ===
pipeline = ImbPipeline([
    ('scaler', StandardScaler()),
    ('smote', SMOTE(random_state=42)),
    ('nn', MLPClassifier(random_state=42, max_iter=1000))
])

# === Define parameter grid ===
param_grid = {
    'nn__hidden_layer_sizes': [(50,), (100,), (50, 50)],
    'nn__alpha': [0.0001, 0.001, 0.01],
    'nn__solver': ['adam', 'lbfgs'],
    'nn__learning_rate': ['constant', 'adaptive']
}

# === Grid Search ===
grid = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    scoring='recall',
    cv=3,
    n_jobs=-1
)

# === Fit model ===
grid.fit(X_train, y_train)

# === Best model ===
best_model = grid.best_estimator_
print(f"✅ Best model parameters: {grid.best_params_}")

# === Predictions ===
y_pred = best_model.predict(X_test)
y_proba = best_model.predict_proba(X_test)[:, 1]

# === Metrics ===
accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
mcc = matthews_corrcoef(y_test, y_pred)
auc = roc_auc_score(y_test, y_proba)
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
specificity = tn / (tn + fp)
f1 = f1_score(y_test, y_pred)

# === Cross-validated Recall ===
cv_recall_scores = cross_val_score(best_model, X_train, y_train, cv=5, scoring='recall')
cv_recall_mean = cv_recall_scores.mean()

# === Save metrics to CSV ===
header = [
    "Model_Title", "Accuracy", "Recall", "Specificity",
    "AUC", "MCC", "CV_Recall_Mean", "F1"
]

row = [
    model_title, accuracy, recall, specificity,
    auc, mcc, cv_recall_mean, f1
]

file_exists = os.path.isfile(output_csv_path)

with open(output_csv_path, mode='a', newline='') as file:
    writer = csv.writer(file)
    if not file_exists:
        writer.writerow(header)
    writer.writerow(row)

print(f"✅ Results saved to {output_csv_path}")

# === Permutation Feature Importance ===
print(f"\n📊 Computing feature importances for: {model_title}")
result = permutation_importance(
    best_model, X_test, y_test, n_repeats=10, random_state=42, scoring='recall'
)
importances = pd.Series(result.importances_mean, index=feature_names)

# === Save feature importances ===
importance_df = pd.DataFrame(
    importances.values.reshape(1, -1), columns=feature_names, index=[model_title]
)

if os.path.exists(feature_importance_csv):
    existing_df = pd.read_csv(feature_importance_csv, index_col=0)
    updated_df = pd.concat([existing_df, importance_df])
else:
    updated_df = importance_df

updated_df.to_csv(feature_importance_csv)
print(f"✅ Feature importances saved to {feature_importance_csv}")


In [ ]:
#xgboost


In [ ]:
#xgboost standard
import csv
import os
import pandas as pd
from xgboost import XGBClassifier
from sklearn.metrics import (
    accuracy_score, recall_score, confusion_matrix,
    roc_auc_score, matthews_corrcoef
)
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.inspection import permutation_importance

# --- Model title for tracking
model_title = "XGBoost_original"

# --- Use raw train/test sets
X_train_input = X_train
y_train_input = y_train
X_test_input  = X_test
y_test_input  = y_test

# --- Initialize XGBoost
model = XGBClassifier(
    use_label_encoder=False,
    eval_metric='logloss',
    random_state=42
)

# --- Train and Predict
model.fit(X_train_input, y_train_input)
y_pred = model.predict(X_test_input)
y_proba = model.predict_proba(X_test_input)[:, 1]

# --- Metrics
accuracy = accuracy_score(y_test_input, y_pred)
recall = recall_score(y_test_input, y_pred)
mcc = matthews_corrcoef(y_test_input, y_pred)
auc = roc_auc_score(y_test_input, y_proba)
tn, fp, fn, tp = confusion_matrix(y_test_input, y_pred).ravel()
specificity = tn / (tn + fp)
f1 = f1_score(y_test_input, y_pred)


# --- Cross-validated recall (on full dataset)
pipeline = make_pipeline(model)
cv_recall_scores = cross_val_score(pipeline, X, y, cv=5, scoring='recall')
cv_recall_mean = cv_recall_scores.mean()

# --- Save model evaluation
header = ["Model_Title", "Accuracy", "Recall", "Specificity", "AUC", "MCC", "CV_Recall_Mean", "F1"]
row = [model_title, accuracy, recall, specificity, auc, mcc, cv_recall_mean, f1]

file_exists = os.path.isfile(output_csv_path)
with open(output_csv_path, mode='a', newline='') as file:
    writer = csv.writer(file)
    if not file_exists:
        writer.writerow(header)
    writer.writerow(row)

print(f"✅ Evaluation metrics saved to {output_csv_path}")

# --- Permutation Feature Importance
print(f"\n📊 Computing permutation importances for: {model_title}")
result = permutation_importance(
    model, X_test_input, y_test_input, n_repeats=10, random_state=42, scoring='precision'
)

importances = pd.Series(result.importances_mean, index=feature_names)

# 🔄 Transpose for saving
importance_df = pd.DataFrame(
    importances.values.reshape(1, -1),
    columns=feature_names,
    index=[model_title]
)

if os.path.exists(feature_importance_csv):
    existing_df = pd.read_csv(feature_importance_csv, index_col=0)
    updated_df = pd.concat([existing_df, importance_df])
else:
    updated_df = importance_df

updated_df.to_csv(feature_importance_csv)
print(f"✅ Feature importances saved to {feature_importance_csv}")

In [ ]:
#xgboost scaled
import csv
import os
import pandas as pd
from sklearn.metrics import (
    accuracy_score, recall_score, confusion_matrix,
    roc_auc_score, matthews_corrcoef, f1_score
)
from sklearn.model_selection import cross_val_score
from sklearn.inspection import permutation_importance
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from xgboost import XGBClassifier

# === Set model title ===
model_title = "XGBoost_Scaled"

# === Define pipeline: Scaling -> XGBoost
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('xgb', XGBClassifier(
        use_label_encoder=False,
        eval_metric='logloss',
        random_state=42,
        scale_pos_weight=1,  # Adjust if there's class imbalance
        n_jobs=-1
    ))
])

# === Fit model ===
pipeline.fit(X_train, y_train)

# === Predictions ===
y_pred = pipeline.predict(X_test)
y_proba = pipeline.predict_proba(X_test)[:, 1]

# === Metrics ===
accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred, pos_label=1)
mcc = matthews_corrcoef(y_test, y_pred)
auc = roc_auc_score(y_test, y_proba)
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
specificity = tn / (tn + fp)
f1 = f1_score(y_test, y_pred)

# === Cross-validated Recall ===
cv_recall_scores = cross_val_score(pipeline, X_train, y_train, cv=5, scoring='recall')
cv_recall_mean = cv_recall_scores.mean()

# === Save metrics to CSV ===
header = [
    "Model_Title", "Accuracy", "Recall", "Specificity",
    "AUC", "MCC", "CV_Recall_Mean", "F1"
]

row = [
    model_title, accuracy, recall, specificity,
    auc, mcc, cv_recall_mean, f1
]

file_exists = os.path.isfile(output_csv_path)

with open(output_csv_path, mode='a', newline='') as file:
    writer = csv.writer(file)
    if not file_exists:
        writer.writerow(header)
    writer.writerow(row)

print(f"✅ Results saved to {output_csv_path}")

# === Permutation Feature Importance ===
print(f"\n📊 Computing feature importances for: {model_title}")
result = permutation_importance(
    pipeline, X_test, y_test, n_repeats=10, random_state=42, scoring='recall'
)
importances = pd.Series(result.importances_mean, index=feature_names)

# === Save feature importances ===
importance_df = pd.DataFrame(
    importances.values.reshape(1, -1), columns=feature_names, index=[model_title]
)

if os.path.exists(feature_importance_csv):
    existing_df = pd.read_csv(feature_importance_csv, index_col=0)
    updated_df = pd.concat([existing_df, importance_df])
else:
    updated_df = importance_df

updated_df.to_csv(feature_importance_csv)
print(f"✅ Feature importances saved to {feature_importance_csv}")


In [ ]:
#xgboost smote
import csv
import os
import pandas as pd
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline  # imblearn's pipeline
from xgboost import XGBClassifier
from sklearn.metrics import (
    accuracy_score, recall_score, confusion_matrix,
    roc_auc_score, matthews_corrcoef, f1_score
)
from sklearn.model_selection import cross_val_score
from sklearn.inspection import permutation_importance

# === Set model title ===
model_title = "XGBoost_SMOTE"

# === Define pipeline: SMOTE -> XGBoost
pipeline = Pipeline([
    ('smote', SMOTE(random_state=42)),
    ('xgb', XGBClassifier(
        use_label_encoder=False,
        eval_metric='logloss',
        random_state=42,
        scale_pos_weight=1,
        n_jobs=-1
    ))
])

# === Fit model ===
pipeline.fit(X_train, y_train)

# === Predictions ===
y_pred = pipeline.predict(X_test)
y_proba = pipeline.predict_proba(X_test)[:, 1]

# === Metrics ===
accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred, pos_label=1)
mcc = matthews_corrcoef(y_test, y_pred)
auc = roc_auc_score(y_test, y_proba)
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
specificity = tn / (tn + fp)
f1 = f1_score(y_test, y_pred)

# === Cross-validated Recall ===
cv_recall_scores = cross_val_score(pipeline, X_train, y_train, cv=5, scoring='recall')
cv_recall_mean = cv_recall_scores.mean()

# === Save metrics to CSV ===
header = [
    "Model_Title", "Accuracy", "Recall", "Specificity",
    "AUC", "MCC", "CV_Recall_Mean", "F1"
]

row = [
    model_title, accuracy, recall, specificity,
    auc, mcc, cv_recall_mean, f1
]

file_exists = os.path.isfile(output_csv_path)

with open(output_csv_path, mode='a', newline='') as file:
    writer = csv.writer(file)
    if not file_exists:
        writer.writerow(header)
    writer.writerow(row)

print(f"✅ Results saved to {output_csv_path}")

# === Permutation Feature Importance ===
print(f"\n📊 Computing feature importances for: {model_title}")
result = permutation_importance(
    pipeline, X_test, y_test, n_repeats=10, random_state=42, scoring='recall'
)
importances = pd.Series(result.importances_mean, index=feature_names)

# === Save feature importances ===
importance_df = pd.DataFrame(
    importances.values.reshape(1, -1), columns=feature_names, index=[model_title]
)

if os.path.exists(feature_importance_csv):
    existing_df = pd.read_csv(feature_importance_csv, index_col=0)
    updated_df = pd.concat([existing_df, importance_df])
else:
    updated_df = importance_df

updated_df.to_csv(feature_importance_csv)
print(f"✅ Feature importances saved to {feature_importance_csv}")


In [ ]:
#xgboost scaled and smote
import csv
import os
import pandas as pd
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline  # imblearn's pipeline
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
from sklearn.metrics import (
    accuracy_score, recall_score, confusion_matrix,
    roc_auc_score, matthews_corrcoef, f1_score
)
from sklearn.model_selection import cross_val_score
from sklearn.inspection import permutation_importance

# === Set model title ===
model_title = "XGBoost_SMOTE_Scaled"

# === Define pipeline: SMOTE -> Scaling -> XGBoost
pipeline = Pipeline([
    ('smote', SMOTE(random_state=42)),
    ('scaler', StandardScaler()),
    ('xgb', XGBClassifier(
        use_label_encoder=False,
        eval_metric='logloss',
        random_state=42,
        scale_pos_weight=1,
        n_jobs=-1
    ))
])

# === Fit model ===
pipeline.fit(X_train, y_train)

# === Predictions ===
y_pred = pipeline.predict(X_test)
y_proba = pipeline.predict_proba(X_test)[:, 1]

# === Metrics ===
accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
mcc = matthews_corrcoef(y_test, y_pred)
auc = roc_auc_score(y_test, y_proba)
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
specificity = tn / (tn + fp)
f1 = f1_score(y_test, y_pred)

# === Cross-validated Recall ===
cv_recall_scores = cross_val_score(pipeline, X_train, y_train, cv=5, scoring='recall')
cv_recall_mean = cv_recall_scores.mean()

# === Save metrics to CSV ===
header = [
    "Model_Title", "Accuracy", "Recall", "Specificity",
    "AUC", "MCC", "CV_Recall_Mean", "F1"
]

row = [
    model_title, accuracy, recall, specificity,
    auc, mcc, cv_recall_mean, f1
]

file_exists = os.path.isfile(output_csv_path)

with open(output_csv_path, mode='a', newline='') as file:
    writer = csv.writer(file)
    if not file_exists:
        writer.writerow(header)
    writer.writerow(row)

print(f"✅ Results saved to {output_csv_path}")

# === Permutation Feature Importance ===
print(f"\n📊 Computing feature importances for: {model_title}")
result = permutation_importance(
    pipeline, X_test, y_test, n_repeats=10, random_state=42, scoring='recall'
)
importances = pd.Series(result.importances_mean, index=feature_names)

# === Save feature importances ===
importance_df = pd.DataFrame(
    importances.values.reshape(1, -1), columns=feature_names, index=[model_title]
)

if os.path.exists(feature_importance_csv):
    existing_df = pd.read_csv(feature_importance_csv, index_col=0)
    updated_df = pd.concat([existing_df, importance_df])
else:
    updated_df = importance_df

updated_df.to_csv(feature_importance_csv)
print(f"✅ Feature importances saved to {feature_importance_csv}")


In [ ]:
#xgboost gridsearch

import csv
import os
import pandas as pd
from xgboost import XGBClassifier
from sklearn.metrics import (
    accuracy_score, recall_score, confusion_matrix,
    roc_auc_score, matthews_corrcoef
)
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.inspection import permutation_importance

# --- Model title
model_title = "XGBoost_original_GridSearch"

# --- Use unscaled, unbalanced data
X_train_input = X_train
y_train_input = y_train
X_test_input  = X_test
y_test_input  = y_test

# --- Define base model and parameter grid
xgb_base = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)

param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [3, 5],
    'learning_rate': [0.05, 0.1],
    'subsample': [0.8, 1.0]
}

grid = GridSearchCV(xgb_base, param_grid, cv=3, scoring='recall', n_jobs=-1)
grid.fit(X_train_input, y_train_input)

# --- Get best model
best_model = grid.best_estimator_
print(f"✅ Best model params: {grid.best_params_}")

# --- Predict with best model
y_pred = best_model.predict(X_test_input)
y_proba = best_model.predict_proba(X_test_input)[:, 1]

# --- Metrics
accuracy = accuracy_score(y_test_input, y_pred)
recall = recall_score(y_test_input, y_pred)
mcc = matthews_corrcoef(y_test_input, y_pred)
auc = roc_auc_score(y_test_input, y_proba)
tn, fp, fn, tp = confusion_matrix(y_test_input, y_pred).ravel()
specificity = tn / (tn + fp)
f1 = f1_score(y_test_input, y_pred)


# --- Cross-validated recall (on full dataset)
pipeline = make_pipeline(best_model)
cv_recall_scores = cross_val_score(pipeline, X, y, cv=5, scoring='recall')
cv_recall_mean = cv_recall_scores.mean()

# --- Save model evaluation
header = ["Model_Title", "Accuracy", "Recall", "Specificity", "AUC", "MCC", "CV_Recall_Mean", "F1"]
row = [model_title, accuracy, recall, specificity, auc, mcc, cv_recall_mean, f1]

file_exists = os.path.isfile(output_csv_path)
with open(output_csv_path, mode='a', newline='') as file:
    writer = csv.writer(file)
    if not file_exists:
        writer.writerow(header)
    writer.writerow(row)

print(f"✅ Evaluation metrics saved to {output_csv_path}")

# --- Permutation Feature Importance
print(f"\n📊 Computing permutation importances for: {model_title}")
result = permutation_importance(
    best_model, X_test_input, y_test_input, n_repeats=10, random_state=42, scoring='precision'
)

importances = pd.Series(result.importances_mean, index=feature_names)

# 🔄 Transpose and save
importance_df = pd.DataFrame(
    importances.values.reshape(1, -1),
    columns=feature_names,
    index=[model_title]
)

if os.path.exists(feature_importance_csv):
    existing_df = pd.read_csv(feature_importance_csv, index_col=0)
    updated_df = pd.concat([existing_df, importance_df])
else:
    updated_df = importance_df

updated_df.to_csv(feature_importance_csv)
print(f"✅ Feature importances saved to {feature_importance_csv}")

In [ ]:
#xgboost scaled grid search 
import csv
import os
import pandas as pd
from xgboost import XGBClassifier
from sklearn.metrics import (
    accuracy_score, recall_score, confusion_matrix,
    roc_auc_score, matthews_corrcoef, f1_score
)
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.inspection import permutation_importance
from sklearn.preprocessing import StandardScaler

# --- Model title
model_title = "XGBoost_scaled_GridSearch"

# --- Scale data
scaler = StandardScaler()
X_train_input = scaler.fit_transform(X_train)
X_test_input = scaler.transform(X_test)
y_train_input = y_train
y_test_input = y_test

# --- Define base model and parameter grid
xgb_base = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)

param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [3, 5],
    'learning_rate': [0.05, 0.1],
    'subsample': [0.8, 1.0]
}

grid = GridSearchCV(xgb_base, param_grid, cv=3, scoring='recall', n_jobs=-1)
grid.fit(X_train_input, y_train_input)

# --- Get best model
best_model = grid.best_estimator_
print(f"✅ Best model params: {grid.best_params_}")

# --- Predict with best model
y_pred = best_model.predict(X_test_input)
y_proba = best_model.predict_proba(X_test_input)[:, 1]

# --- Metrics
accuracy = accuracy_score(y_test_input, y_pred)
recall = recall_score(y_test_input, y_pred)
mcc = matthews_corrcoef(y_test_input, y_pred)
auc = roc_auc_score(y_test_input, y_proba)
tn, fp, fn, tp = confusion_matrix(y_test_input, y_pred).ravel()
specificity = tn / (tn + fp)
f1 = f1_score(y_test_input, y_pred)

# --- Cross-validated recall (on full unscaled dataset for fair comp)
pipeline = make_pipeline(StandardScaler(), best_model)
cv_recall_scores = cross_val_score(pipeline, X, y, cv=5, scoring='recall')
cv_recall_mean = cv_recall_scores.mean()

# --- Save model evaluation
header = ["Model_Title", "Accuracy", "Recall", "Specificity", "AUC", "MCC", "CV_Recall_Mean", "F1"]
row = [model_title, accuracy, recall, specificity, auc, mcc, cv_recall_mean, f1]

file_exists = os.path.isfile(output_csv_path)
with open(output_csv_path, mode='a', newline='') as file:
    writer = csv.writer(file)
    if not file_exists:
        writer.writerow(header)
    writer.writerow(row)

print(f"✅ Evaluation metrics saved to {output_csv_path}")

# --- Permutation Feature Importance
print(f"\n📊 Computing permutation importances for: {model_title}")
result = permutation_importance(
    best_model, X_test_input, y_test_input, n_repeats=10, random_state=42, scoring='precision'
)

importances = pd.Series(result.importances_mean, index=feature_names)

# 🔄 Transpose and save
importance_df = pd.DataFrame(
    importances.values.reshape(1, -1),
    columns=feature_names,
    index=[model_title]
)

if os.path.exists(feature_importance_csv):
    existing_df = pd.read_csv(feature_importance_csv, index_col=0)
    updated_df = pd.concat([existing_df, importance_df])
else:
    updated_df = importance_df

updated_df.to_csv(feature_importance_csv)
print(f"✅ Feature importances saved to {feature_importance_csv}")

In [ ]:
#xgboost smote grid search
import csv
import os
import pandas as pd
from xgboost import XGBClassifier
from sklearn.metrics import (
    accuracy_score, recall_score, confusion_matrix,
    roc_auc_score, matthews_corrcoef
)
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.inspection import permutation_importance
from sklearn.preprocessing import StandardScaler

# 🔁 Update this for each run
model_title = "XGBoost_gridsearch_scaled_SMOTE"  # Change title for each run

# 🧠 Define your input data variant
X_train_input = X_train_resampled   # e.g., X_train, X_train_scaled, X_train_resampled
y_train_input = y_train_resampled
X_test_input  = X_test_scaled       # e.g., X_test or X_test_scaled
y_test_input  = y_test

# Step 1: GridSearchCV
xgb_base = XGBClassifier(
    use_label_encoder=False,
    eval_metric='logloss',
    random_state=42,
    scale_pos_weight=1  # adjust if imbalance is extreme (or skip if using SMOTE)
)

param_grid = {
    'n_estimators': [50, 100],
    'max_depth': [3, 5],
    'learning_rate': [0.01, 0.1],
    'subsample': [0.8, 1.0]
}

grid_search = GridSearchCV(
    estimator=xgb_base,
    param_grid=param_grid,
    scoring='precision',
    cv=5,
    n_jobs=-1,
    verbose=2
)

# Step 2: Train model and predict
grid_search.fit(X_train_input, y_train_input)
best_model = grid_search.best_estimator_
print(f"✅ Best XGBoost Params: {grid_search.best_params_}")

y_pred = best_model.predict(X_test_input)
y_proba = best_model.predict_proba(X_test_input)[:, 1]

# Step 3: Evaluation metrics
accuracy = accuracy_score(y_test_input, y_pred)
recall = recall_score(y_test_input, y_pred)
mcc = matthews_corrcoef(y_test_input, y_pred)
auc = roc_auc_score(y_test_input, y_proba)
tn, fp, fn, tp = confusion_matrix(y_test_input, y_pred).ravel()
specificity = tn / (tn + fp)
f1 = f1_score(y_test_input, y_pred)

# Step 4: Cross-validated recall (on full dataset, no resampling)
pipeline = make_pipeline(StandardScaler(), best_model)
cv_recall_scores = cross_val_score(pipeline, X, y, cv=5, scoring='recall')
cv_recall_mean = cv_recall_scores.mean()

# Step 5: Save metrics to CSV
header = ["Model_Title", "Accuracy", "Recall", "Specificity", "AUC", "MCC", "CV_Recall_Mean", "F1"]
row = [model_title, accuracy, recall, specificity, auc, mcc, cv_recall_mean, f1]

file_exists = os.path.isfile(output_csv_path)
with open(output_csv_path, mode='a', newline='') as file:
    writer = csv.writer(file)
    if not file_exists:
        writer.writerow(header)
    writer.writerow(row)

print(f"✅ Evaluation metrics saved to {output_csv_path}")

# Step 6: Feature importance via permutation
print(f"\n📊 Computing permutation importances for: {model_title}")
result = permutation_importance(
    best_model, X_test_input, y_test_input, n_repeats=10,
    random_state=42, scoring='precision'
)

importances = pd.Series(result.importances_mean, index=feature_names)

importance_df = pd.DataFrame(
    importances.values.reshape(1, -1), columns=feature_names, index=[model_title]
)

# Save to feature importance matrix
if os.path.exists(feature_importance_csv):
    existing_df = pd.read_csv(feature_importance_csv, index_col=0)
    updated_df = pd.concat([existing_df, importance_df])
else:
    updated_df = importance_df

updated_df.to_csv(feature_importance_csv)
print(f"✅ Feature importances saved to {feature_importance_csv}")

In [ ]:
#xgboost scaled, smote and gridsearch 
import csv
import os
import pandas as pd
from sklearn.metrics import (
    accuracy_score, recall_score, confusion_matrix,
    roc_auc_score, matthews_corrcoef, f1_score
)
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.inspection import permutation_importance
from sklearn.preprocessing import StandardScaler
from imblearn.pipeline import Pipeline  # imblearn pipeline for SMOTE
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier

# === Set model title ===
model_title = "XGBoost_SMOTE_Scaled_GridSearch"

# === Define pipeline
pipeline = Pipeline([
    ('smote', SMOTE(random_state=42)),
    ('scaler', StandardScaler()),
    ('xgb', XGBClassifier(
        use_label_encoder=False,
        eval_metric='logloss',
        random_state=42,
        n_jobs=-1
    ))
])

# === Define parameter grid
param_grid = {
    'xgb__n_estimators': [100, 200],
    'xgb__max_depth': [3, 5],
    'xgb__learning_rate': [0.01, 0.1],
    'xgb__subsample': [0.8, 1.0]
}

# === GridSearchCV
grid = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    scoring='recall',
    cv=3,
    n_jobs=-1,
    verbose=1
)

grid.fit(X_train, y_train)

# === Best model
best_model = grid.best_estimator_
print(f"✅ Best model parameters: {grid.best_params_}")

# === Predictions
y_pred = best_model.predict(X_test)
y_proba = best_model.predict_proba(X_test)[:, 1]

# === Metrics
accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
mcc = matthews_corrcoef(y_test, y_pred)
auc = roc_auc_score(y_test, y_proba)
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
specificity = tn / (tn + fp)
f1 = f1_score(y_test, y_pred)

# === Cross-validated Recall
cv_recall_scores = cross_val_score(best_model, X_train, y_train, cv=5, scoring='recall')
cv_recall_mean = cv_recall_scores.mean()

# === Save metrics to CSV
header = [
    "Model_Title", "Accuracy", "Recall", "Specificity",
    "AUC", "MCC", "CV_Recall_Mean", "F1"
]

row = [
    model_title, accuracy, recall, specificity,
    auc, mcc, cv_recall_mean, f1
]

file_exists = os.path.isfile(output_csv_path)

with open(output_csv_path, mode='a', newline='') as file:
    writer = csv.writer(file)
    if not file_exists:
        writer.writerow(header)
    writer.writerow(row)

print(f"✅ Results saved to {output_csv_path}")

# === Permutation Feature Importance
print(f"\n📊 Computing feature importances for: {model_title}")
result = permutation_importance(
    best_model, X_test, y_test, n_repeats=10, random_state=42, scoring='recall'
)
importances = pd.Series(result.importances_mean, index=feature_names)

# === Save feature importances
importance_df = pd.DataFrame(
    importances.values.reshape(1, -1), columns=feature_names, index=[model_title]
)

if os.path.exists(feature_importance_csv):
    existing_df = pd.read_csv(feature_importance_csv, index_col=0)
    updated_df = pd.concat([existing_df, importance_df])
else:
    updated_df = importance_df

updated_df.to_csv(feature_importance_csv)
print(f"✅ Feature importances saved to {feature_importance_csv}")


#### Naive Bayers

In [ ]:
#Naive Bayers standard 
import csv
import os
import pandas as pd
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import (
    accuracy_score, recall_score, confusion_matrix,
    roc_auc_score, matthews_corrcoef, f1_score
)
from sklearn.model_selection import cross_val_score
from sklearn.inspection import permutation_importance

# === Set model title ===
model_title = "NaiveBayes_original"  # Updated model title

# === Train the Naive Bayes model ===
model = GaussianNB()
model.fit(X_train, y_train)

# === Predictions ===
y_pred = model.predict(X_test)
y_proba = model.predict_proba(X_test)[:, 1]

# === Metrics ===
accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred, pos_label=1)
mcc = matthews_corrcoef(y_test, y_pred)
auc = roc_auc_score(y_test, y_proba)
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
specificity = tn / (tn + fp)
f1 = f1_score(y_test, y_pred)

# === Cross-validated Recall ===
cv_recall_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='recall')
cv_recall_mean = cv_recall_scores.mean()

# === Save metrics to CSV ===
header = [
    "Model_Title", "Accuracy", "Recall", "Specificity",
    "AUC", "MCC", "CV_Recall_Mean", "F1"
]

row = [
    model_title, accuracy, recall, specificity,
    auc, mcc, cv_recall_mean, f1
]

file_exists = os.path.isfile(output_csv_path)

with open(output_csv_path, mode='a', newline='') as file:
    writer = csv.writer(file)
    if not file_exists:
        writer.writerow(header)
    writer.writerow(row)

print(f"✅ Results saved to {output_csv_path}")

# === Permutation Feature Importance ===
print(f"\n📊 Computing feature importances for: {model_title}")
result = permutation_importance(
    model, X_test, y_test, n_repeats=10, random_state=42, scoring='recall'
)
importances = pd.Series(result.importances_mean, index=feature_names)

# === Save feature importances ===
importance_df = pd.DataFrame(
    importances.values.reshape(1, -1), columns=feature_names, index=[model_title]
)

if os.path.exists(feature_importance_csv):
    existing_df = pd.read_csv(feature_importance_csv, index_col=0)
    updated_df = pd.concat([existing_df, importance_df])
else:
    updated_df = importance_df

updated_df.to_csv(feature_importance_csv)
print(f"✅ Feature importances saved to {feature_importance_csv}")

In [ ]:
#NB Scaled
import csv
import os
import pandas as pd
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import (
    accuracy_score, recall_score, confusion_matrix,
    roc_auc_score, matthews_corrcoef, f1_score
)
from sklearn.model_selection import cross_val_score
from sklearn.inspection import permutation_importance
from sklearn.preprocessing import StandardScaler

# === Set model title ===
model_title = "NaiveBayes_Scaled"  # Updated model title

# === Scale features ===
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# === Train the Naive Bayes model ===
model = GaussianNB()
model.fit(X_train_scaled, y_train)

# === Predictions ===
y_pred = model.predict(X_test_scaled)
y_proba = model.predict_proba(X_test_scaled)[:, 1]

# === Metrics ===
accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred, pos_label=1)
mcc = matthews_corrcoef(y_test, y_pred)
auc = roc_auc_score(y_test, y_proba)
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
specificity = tn / (tn + fp)
f1 = f1_score(y_test, y_pred)

# === Cross-validated Recall (on scaled data) ===
cv_recall_scores = cross_val_score(model, X_train_scaled, y_train, cv=5, scoring='recall')
cv_recall_mean = cv_recall_scores.mean()

# === Save metrics to CSV ===
header = [
    "Model_Title", "Accuracy", "Recall", "Specificity",
    "AUC", "MCC", "CV_Recall_Mean", "F1"
]

row = [
    model_title, accuracy, recall, specificity,
    auc, mcc, cv_recall_mean, f1
]

file_exists = os.path.isfile(output_csv_path)

with open(output_csv_path, mode='a', newline='') as file:
    writer = csv.writer(file)
    if not file_exists:
        writer.writerow(header)
    writer.writerow(row)

print(f"✅ Results saved to {output_csv_path}")

# === Permutation Feature Importance ===
print(f"\n📊 Computing feature importances for: {model_title}")
result = permutation_importance(
    model, X_test_scaled, y_test, n_repeats=10, random_state=42, scoring='recall'
)
importances = pd.Series(result.importances_mean, index=feature_names)

# === Save feature importances ===
importance_df = pd.DataFrame(
    importances.values.reshape(1, -1), columns=feature_names, index=[model_title]
)

if os.path.exists(feature_importance_csv):
    existing_df = pd.read_csv(feature_importance_csv, index_col=0)
    updated_df = pd.concat([existing_df, importance_df])
else:
    updated_df = importance_df

updated_df.to_csv(feature_importance_csv)
print(f"✅ Feature importances saved to {feature_importance_csv}")

In [ ]:
#NB with smote
import csv
import os
import pandas as pd
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import (
    accuracy_score, recall_score, confusion_matrix,
    roc_auc_score, matthews_corrcoef, f1_score
)
from sklearn.model_selection import cross_val_score
from sklearn.inspection import permutation_importance
from imblearn.over_sampling import SMOTE  # ✅ Added SMOTE

# === Set model title ===
model_title = "NaiveBayes_SMOTE"

# === Apply SMOTE ===
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# === Train the Naive Bayes model ===
model = GaussianNB()
model.fit(X_train_resampled, y_train_resampled)

# === Predictions ===
y_pred = model.predict(X_test)
y_proba = model.predict_proba(X_test)[:, 1]

# === Metrics ===
accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred, pos_label=1)
mcc = matthews_corrcoef(y_test, y_pred)
auc = roc_auc_score(y_test, y_proba)
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
specificity = tn / (tn + fp)
f1 = f1_score(y_test, y_pred)

# === Cross-validated Recall (on original data for comparison) ===
cv_recall_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='recall')
cv_recall_mean = cv_recall_scores.mean()

# === Save metrics to CSV ===
header = [
    "Model_Title", "Accuracy", "Recall", "Specificity",
    "AUC", "MCC", "CV_Recall_Mean", "F1"
]

row = [
    model_title, accuracy, recall, specificity,
    auc, mcc, cv_recall_mean, f1
]

file_exists = os.path.isfile(output_csv_path)

with open(output_csv_path, mode='a', newline='') as file:
    writer = csv.writer(file)
    if not file_exists:
        writer.writerow(header)
    writer.writerow(row)

print(f"✅ Results saved to {output_csv_path}")

# === Permutation Feature Importance ===
print(f"\n📊 Computing feature importances for: {model_title}")
result = permutation_importance(
    model, X_test, y_test, n_repeats=10, random_state=42, scoring='recall'
)
importances = pd.Series(result.importances_mean, index=feature_names)

# === Save feature importances ===
importance_df = pd.DataFrame(
    importances.values.reshape(1, -1), columns=feature_names, index=[model_title]
)

if os.path.exists(feature_importance_csv):
    existing_df = pd.read_csv(feature_importance_csv, index_col=0)
    updated_df = pd.concat([existing_df, importance_df])
else:
    updated_df = importance_df

updated_df.to_csv(feature_importance_csv)
print(f"✅ Feature importances saved to {feature_importance_csv}")


In [ ]:
#NB Gridsearch 
import csv
import os
import pandas as pd
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import (
    accuracy_score, recall_score, confusion_matrix,
    roc_auc_score, matthews_corrcoef, f1_score
)
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.inspection import permutation_importance

# === Set model title ===
model_title = "NaiveBayes_GridSearch"

# === Define parameter grid ===
param_grid = {
    'var_smoothing': [1e-9, 1e-8, 1e-7, 1e-6]
}

# === GridSearchCV ===
grid = GridSearchCV(
    estimator=GaussianNB(),
    param_grid=param_grid,
    scoring='recall',
    cv=3,
    n_jobs=-1
)
grid.fit(X_train, y_train)

# === Best model ===
model = grid.best_estimator_
print(f"✅ Best model parameters: {grid.best_params_}")

# === Predictions ===
y_pred = model.predict(X_test)
y_proba = model.predict_proba(X_test)[:, 1]

# === Metrics ===
accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred, pos_label=1)
mcc = matthews_corrcoef(y_test, y_pred)
auc = roc_auc_score(y_test, y_proba)
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
specificity = tn / (tn + fp)
f1 = f1_score(y_test, y_pred)

# === Cross-validated Recall ===
cv_recall_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='recall')
cv_recall_mean = cv_recall_scores.mean()

# === Save metrics to CSV ===
header = [
    "Model_Title", "Accuracy", "Recall", "Specificity",
    "AUC", "MCC", "CV_Recall_Mean", "F1"
]

row = [
    model_title, accuracy, recall, specificity,
    auc, mcc, cv_recall_mean, f1
]

file_exists = os.path.isfile(output_csv_path)

with open(output_csv_path, mode='a', newline='') as file:
    writer = csv.writer(file)
    if not file_exists:
        writer.writerow(header)
    writer.writerow(row)

print(f"✅ Results saved to {output_csv_path}")

# === Permutation Feature Importance ===
print(f"\n📊 Computing feature importances for: {model_title}")
result = permutation_importance(
    model, X_test, y_test, n_repeats=10, random_state=42, scoring='recall'
)
importances = pd.Series(result.importances_mean, index=feature_names)

# === Save feature importances ===
importance_df = pd.DataFrame(
    importances.values.reshape(1, -1), columns=feature_names, index=[model_title]
)

if os.path.exists(feature_importance_csv):
    existing_df = pd.read_csv(feature_importance_csv, index_col=0)
    updated_df = pd.concat([existing_df, importance_df])
else:
    updated_df = importance_df

updated_df.to_csv(feature_importance_csv)
print(f"✅ Feature importances saved to {feature_importance_csv}")


In [ ]:
#NB with smote and gridsearch
import csv
import os
import pandas as pd
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import (
    accuracy_score, recall_score, confusion_matrix,
    roc_auc_score, matthews_corrcoef, f1_score
)
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.inspection import permutation_importance
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE

# === Set model title ===
model_title = "NaiveBayes_SMOTE_GridSearch"

# === Define parameter grid ===
param_grid = {
    'model__var_smoothing': [1e-9, 1e-8, 1e-7, 1e-6]
}

# === Build pipeline: SMOTE + Naive Bayes ===
pipeline = Pipeline([
    ('smote', SMOTE(random_state=42)),
    ('model', GaussianNB())
])

# === Grid Search ===
grid = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    scoring='recall',
    cv=3,
    n_jobs=-1
)
grid.fit(X_train, y_train)

# === Best model ===
model = grid.best_estimator_
print(f"✅ Best model parameters: {grid.best_params_}")

# === Predictions ===
y_pred = model.predict(X_test)
y_proba = model.predict_proba(X_test)[:, 1]

# === Metrics ===
accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
mcc = matthews_corrcoef(y_test, y_pred)
auc = roc_auc_score(y_test, y_proba)
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
specificity = tn / (tn + fp)
f1 = f1_score(y_test, y_pred)

# === Cross-validated Recall ===
cv_recall_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='recall')
cv_recall_mean = cv_recall_scores.mean()

# === Save metrics to CSV ===
header = [
    "Model_Title", "Accuracy", "Recall", "Specificity",
    "AUC", "MCC", "CV_Recall_Mean", "F1"
]

row = [
    model_title, accuracy, recall, specificity,
    auc, mcc, cv_recall_mean, f1
]

file_exists = os.path.isfile(output_csv_path)

with open(output_csv_path, mode='a', newline='') as file:
    writer = csv.writer(file)
    if not file_exists:
        writer.writerow(header)
    writer.writerow(row)

print(f"✅ Results saved to {output_csv_path}")

# === Permutation Feature Importance ===
print(f"\n📊 Computing feature importances for: {model_title}")
result = permutation_importance(
    model, X_test, y_test, n_repeats=10, random_state=42, scoring='recall'
)
importances = pd.Series(result.importances_mean, index=feature_names)

# === Save feature importances ===
importance_df = pd.DataFrame(
    importances.values.reshape(1, -1), columns=feature_names, index=[model_title]
)

if os.path.exists(feature_importance_csv):
    existing_df = pd.read_csv(feature_importance_csv, index_col=0)
    updated_df = pd.concat([existing_df, importance_df])
else:
    updated_df = importance_df

updated_df.to_csv(feature_importance_csv)
print(f"✅ Feature importances saved to {feature_importance_csv}")


In [ ]:
#nb  scaled and gridsearch
import csv
import os
import pandas as pd
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import (
    accuracy_score, recall_score, confusion_matrix,
    roc_auc_score, matthews_corrcoef, f1_score
)
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.inspection import permutation_importance
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# === Set model title ===
model_title = "NaiveBayes_Scaled_GridSearch"

# === Define parameter grid ===
param_grid = {
    'model__var_smoothing': [1e-9, 1e-8, 1e-7, 1e-6]
}

# === Build pipeline: Scaling + Naive Bayes ===
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('model', GaussianNB())
])

# === Grid Search ===
grid = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    scoring='recall',
    cv=3,
    n_jobs=-1
)
grid.fit(X_train, y_train)

# === Best model ===
model = grid.best_estimator_
print(f"✅ Best model parameters: {grid.best_params_}")

# === Predictions ===
y_pred = model.predict(X_test)
y_proba = model.predict_proba(X_test)[:, 1]

# === Metrics ===
accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
mcc = matthews_corrcoef(y_test, y_pred)
auc = roc_auc_score(y_test, y_proba)
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
specificity = tn / (tn + fp)
f1 = f1_score(y_test, y_pred)

# === Cross-validated Recall ===
cv_recall_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='recall')
cv_recall_mean = cv_recall_scores.mean()

# === Save metrics to CSV ===
header = [
    "Model_Title", "Accuracy", "Recall", "Specificity",
    "AUC", "MCC", "CV_Recall_Mean", "F1"
]

row = [
    model_title, accuracy, recall, specificity,
    auc, mcc, cv_recall_mean, f1
]

file_exists = os.path.isfile(output_csv_path)

with open(output_csv_path, mode='a', newline='') as file:
    writer = csv.writer(file)
    if not file_exists:
        writer.writerow(header)
    writer.writerow(row)

print(f"✅ Results saved to {output_csv_path}")

# === Permutation Feature Importance ===
print(f"\n📊 Computing feature importances for: {model_title}")
result = permutation_importance(
    model, X_test, y_test, n_repeats=10, random_state=42, scoring='recall'
)
importances = pd.Series(result.importances_mean, index=feature_names)

# === Save feature importances ===
importance_df = pd.DataFrame(
    importances.values.reshape(1, -1), columns=feature_names, index=[model_title]
)

if os.path.exists(feature_importance_csv):
    existing_df = pd.read_csv(feature_importance_csv, index_col=0)
    updated_df = pd.concat([existing_df, importance_df])
else:
    updated_df = importance_df

updated_df.to_csv(feature_importance_csv)
print(f"✅ Feature importances saved to {feature_importance_csv}")


In [ ]:
#nb with scaling smote and gridsearch
import csv
import os
import pandas as pd
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import (
    accuracy_score, recall_score, confusion_matrix,
    roc_auc_score, matthews_corrcoef, f1_score
)
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.inspection import permutation_importance
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline

# === Set model title ===
model_title = "NaiveBayes_Scaled_SMOTE_GridSearch"

# === Define pipeline: Scaling -> SMOTE -> Naive Bayes ===
pipeline = ImbPipeline([
    ('scaler', StandardScaler()),
    ('smote', SMOTE(random_state=42)),
    ('model', GaussianNB())
])

# === Define parameter grid ===
param_grid = {
    'model__var_smoothing': [1e-9, 1e-8, 1e-7, 1e-6]
}

# === Grid Search ===
grid = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    scoring='recall',
    cv=3,
    n_jobs=-1
)
grid.fit(X_train, y_train)

# === Best model ===
best_model = grid.best_estimator_
print(f"✅ Best model parameters: {grid.best_params_}")

# === Predictions ===
y_pred = best_model.predict(X_test)
y_proba = best_model.predict_proba(X_test)[:, 1]

# === Metrics ===
accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
mcc = matthews_corrcoef(y_test, y_pred)
auc = roc_auc_score(y_test, y_proba)
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
specificity = tn / (tn + fp)
f1 = f1_score(y_test, y_pred)

# === Cross-validated Recall ===
cv_recall_scores = cross_val_score(best_model, X_train, y_train, cv=5, scoring='recall')
cv_recall_mean = cv_recall_scores.mean()

# === Save metrics to CSV ===
header = [
    "Model_Title", "Accuracy", "Recall", "Specificity",
    "AUC", "MCC", "CV_Recall_Mean", "F1"
]

row = [
    model_title, accuracy, recall, specificity,
    auc, mcc, cv_recall_mean, f1
]

file_exists = os.path.isfile(output_csv_path)

with open(output_csv_path, mode='a', newline='') as file:
    writer = csv.writer(file)
    if not file_exists:
        writer.writerow(header)
    writer.writerow(row)

print(f"✅ Results saved to {output_csv_path}")

# === Permutation Feature Importance ===
print(f"\n📊 Computing feature importances for: {model_title}")
result = permutation_importance(
    best_model, X_test, y_test, n_repeats=10, random_state=42, scoring='recall'
)
importances = pd.Series(result.importances_mean, index=feature_names)

# === Save feature importances ===
importance_df = pd.DataFrame(
    importances.values.reshape(1, -1), columns=feature_names, index=[model_title]
)

if os.path.exists(feature_importance_csv):
    existing_df = pd.read_csv(feature_importance_csv, index_col=0)
    updated_df = pd.concat([existing_df, importance_df])
else:
    updated_df = importance_df

updated_df.to_csv(feature_importance_csv)
print(f"✅ Feature importances saved to {feature_importance_csv}")
